# 第十二章：并发编程
 对于并发编程, Python有多种长期支持的方法, 包括多线程, 调用子进程, 以及各种各样的关于生成器函数的技巧.
这一章将会给出并发编程各种方面的技巧, 包括通用的多线程技术以及并行计算的实现方法.

## 12.1 启动与停止线程


### 问题


你要为需要并发执行的代码创建/销毁线程

### 解决方案


threading 库可以在单独的线程中执行任何的在 Python 中可以调用的对象。你可以创建一个 Thread 对象并将你要执行的对象以 target 参数的形式提供给该对象。
下面是一个简单的例子：

In [ ]:
#
 
C
o
d
e
 
t
o
 
e
x
e
c
u
t
e
 
i
n
 
a
n
 
i
n
d
e
p
e
n
d
e
n
t
 
t
h
r
e
a
d


i
m
p
o
r
t
 
t
i
m
e


d
e
f
 
c
o
u
n
t
d
o
w
n
(
n
)
:


 
 
 
 
w
h
i
l
e
 
n
 
>
 
0
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
T
-
m
i
n
u
s
'
,
 
n
)


 
 
 
 
 
 
 
 
n
 
-
=
 
1


 
 
 
 
 
 
 
 
t
i
m
e
.
s
l
e
e
p
(
5
)




#
 
C
r
e
a
t
e
 
a
n
d
 
l
a
u
n
c
h
 
a
 
t
h
r
e
a
d


f
r
o
m
 
t
h
r
e
a
d
i
n
g
 
i
m
p
o
r
t
 
T
h
r
e
a
d


t
 
=
 
T
h
r
e
a
d
(
t
a
r
g
e
t
=
c
o
u
n
t
d
o
w
n
,
 
a
r
g
s
=
(
1
0
,
)
)


t
.
s
t
a
r
t
(
)



当你创建好一个线程对象后，该对象并不会立即执行，除非你调用它的 start() 方法（当你调用 start() 方法时，它会调用你传递进来的函数，并把你传递进来的参数传递给该函数）。Python中的线程会在一个单独的系统级线程中执行（比如说一个 POSIX 线程或者一个 Windows 线程），这些线程将由操作系统来全权管理。线程一旦启动，将独立执行直到目标函数返回。你可以查询一个线程对象的状态，看它是否还在执行：

In [ ]:
i
f
 
t
.
i
s
_
a
l
i
v
e
(
)
:


 
 
 
 
p
r
i
n
t
(
'
S
t
i
l
l
 
r
u
n
n
i
n
g
'
)


e
l
s
e
:


 
 
 
 
p
r
i
n
t
(
'
C
o
m
p
l
e
t
e
d
'
)



你也可以将一个线程加入到当前线程，并等待它终止：

In [ ]:
t
.
j
o
i
n
(
)



Python解释器直到所有线程都终止前仍保持运行。对于需要长时间运行的线程或者需要一直运行的后台任务，你应当考虑使用后台线程。
例如：

In [ ]:
t
 
=
 
T
h
r
e
a
d
(
t
a
r
g
e
t
=
c
o
u
n
t
d
o
w
n
,
 
a
r
g
s
=
(
1
0
,
)
,
 
d
a
e
m
o
n
=
T
r
u
e
)


t
.
s
t
a
r
t
(
)



后台线程无法等待，不过，这些线程会在主线程终止时自动销毁。
除了如上所示的两个操作，并没有太多可以对线程做的事情。你无法结束一个线程，无法给它发送信号，无法调整它的调度，也无法执行其他高级操作。如果需要这些特性，你需要自己添加。比如说，如果你需要终止线程，那么这个线程必须通过编程在某个特定点轮询来退出。你可以像下边这样把线程放入一个类中：

In [ ]:
c
l
a
s
s
 
C
o
u
n
t
d
o
w
n
T
a
s
k
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
r
u
n
n
i
n
g
 
=
 
T
r
u
e




 
 
 
 
d
e
f
 
t
e
r
m
i
n
a
t
e
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
r
u
n
n
i
n
g
 
=
 
F
a
l
s
e




 
 
 
 
d
e
f
 
r
u
n
(
s
e
l
f
,
 
n
)
:


 
 
 
 
 
 
 
 
w
h
i
l
e
 
s
e
l
f
.
_
r
u
n
n
i
n
g
 
a
n
d
 
n
 
>
 
0
:


 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
T
-
m
i
n
u
s
'
,
 
n
)


 
 
 
 
 
 
 
 
 
 
 
 
n
 
-
=
 
1


 
 
 
 
 
 
 
 
 
 
 
 
t
i
m
e
.
s
l
e
e
p
(
5
)




c
 
=
 
C
o
u
n
t
d
o
w
n
T
a
s
k
(
)


t
 
=
 
T
h
r
e
a
d
(
t
a
r
g
e
t
=
c
.
r
u
n
,
 
a
r
g
s
=
(
1
0
,
)
)


t
.
s
t
a
r
t
(
)


c
.
t
e
r
m
i
n
a
t
e
(
)
 
#
 
S
i
g
n
a
l
 
t
e
r
m
i
n
a
t
i
o
n


t
.
j
o
i
n
(
)
 
 
 
 
 
 
#
 
W
a
i
t
 
f
o
r
 
a
c
t
u
a
l
 
t
e
r
m
i
n
a
t
i
o
n
 
(
i
f
 
n
e
e
d
e
d
)



如果线程执行一些像I/O这样的阻塞操作，那么通过轮询来终止线程将使得线程之间的协调变得非常棘手。比如，如果一个线程一直阻塞在一个I/O操作上，它就永远无法返回，也就无法检查自己是否已经被结束了。要正确处理这些问题，你需要利用超时循环来小心操作线程。
例子如下：

In [ ]:
c
l
a
s
s
 
I
O
T
a
s
k
:


 
 
 
 
d
e
f
 
t
e
r
m
i
n
a
t
e
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
r
u
n
n
i
n
g
 
=
 
F
a
l
s
e




 
 
 
 
d
e
f
 
r
u
n
(
s
e
l
f
,
 
s
o
c
k
)
:


 
 
 
 
 
 
 
 
#
 
s
o
c
k
 
i
s
 
a
 
s
o
c
k
e
t


 
 
 
 
 
 
 
 
s
o
c
k
.
s
e
t
t
i
m
e
o
u
t
(
5
)
 
 
 
 
 
 
 
 
#
 
S
e
t
 
t
i
m
e
o
u
t
 
p
e
r
i
o
d


 
 
 
 
 
 
 
 
w
h
i
l
e
 
s
e
l
f
.
_
r
u
n
n
i
n
g
:


 
 
 
 
 
 
 
 
 
 
 
 
#
 
P
e
r
f
o
r
m
 
a
 
b
l
o
c
k
i
n
g
 
I
/
O
 
o
p
e
r
a
t
i
o
n
 
w
/
 
t
i
m
e
o
u
t


 
 
 
 
 
 
 
 
 
 
 
 
t
r
y
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
d
a
t
a
 
=
 
s
o
c
k
.
r
e
c
v
(
8
1
9
2
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
b
r
e
a
k


 
 
 
 
 
 
 
 
 
 
 
 
e
x
c
e
p
t
 
s
o
c
k
e
t
.
t
i
m
e
o
u
t
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
c
o
n
t
i
n
u
e


 
 
 
 
 
 
 
 
 
 
 
 
#
 
C
o
n
t
i
n
u
e
d
 
p
r
o
c
e
s
s
i
n
g


 
 
 
 
 
 
 
 
 
 
 
 
.
.
.


 
 
 
 
 
 
 
 
#
 
T
e
r
m
i
n
a
t
e
d


 
 
 
 
 
 
 
 
r
e
t
u
r
n



### 讨论


由于全局解释锁（GIL）的原因，Python 的线程被限制到同一时刻只允许一个线程执行这样一个执行模型。所以，Python 的线程更适用于处理I/O和其他需要并发执行的阻塞操作（比如等待I/O、等待从数据库获取数据等等），而不是需要多处理器并行的计算密集型任务。

有时你会看到下边这种通过继承 Thread 类来实现的线程：

In [ ]:
f
r
o
m
 
t
h
r
e
a
d
i
n
g
 
i
m
p
o
r
t
 
T
h
r
e
a
d




c
l
a
s
s
 
C
o
u
n
t
d
o
w
n
T
h
r
e
a
d
(
T
h
r
e
a
d
)
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
n
)
:


 
 
 
 
 
 
 
 
s
u
p
e
r
(
)
.
_
_
i
n
i
t
_
_
(
)


 
 
 
 
 
 
 
 
s
e
l
f
.
n
 
=
 
n


 
 
 
 
d
e
f
 
r
u
n
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
w
h
i
l
e
 
s
e
l
f
.
n
 
>
 
0
:




 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
T
-
m
i
n
u
s
'
,
 
s
e
l
f
.
n
)


 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
n
 
-
=
 
1


 
 
 
 
 
 
 
 
 
 
 
 
t
i
m
e
.
s
l
e
e
p
(
5
)




c
 
=
 
C
o
u
n
t
d
o
w
n
T
h
r
e
a
d
(
5
)


c
.
s
t
a
r
t
(
)



尽管这样也可以工作，但这使得你的代码依赖于 threading 库，所以你的这些代码只能在线程上下文中使用。上文所写的那些代码、函数都是与 threading 库无关的，这样就使得这些代码可以被用在其他的上下文中，可能与线程有关，也可能与线程无关。比如，你可以通过 multiprocessing 模块在一个单独的进程中执行你的代码：

In [ ]:
i
m
p
o
r
t
 
m
u
l
t
i
p
r
o
c
e
s
s
i
n
g


c
 
=
 
C
o
u
n
t
d
o
w
n
T
a
s
k
(
5
)


p
 
=
 
m
u
l
t
i
p
r
o
c
e
s
s
i
n
g
.
P
r
o
c
e
s
s
(
t
a
r
g
e
t
=
c
.
r
u
n
)


p
.
s
t
a
r
t
(
)



再次重申，这段代码仅适用于 CountdownTask 类是以独立于实际的并发手段（多线程、多进程等等）实现的情况。

## 12.2 判断线程是否已经启动


### 问题


你已经启动了一个线程，但是你想知道它是不是真的已经开始运行了。

### 解决方案


线程的一个关键特性是每个线程都是独立运行且状态不可预测。如果程序中的其他线程需要通过判断某个线程的状态来确定自己下一步的操作，这时线程同步问题就会变得非常棘手。为了解决这些问题，我们需要使用 threading 库中的 Event 对象。
Event 对象包含一个可由线程设置的信号标志，它允许线程等待某些事件的发生。在初始情况下，event 对象中的信号标志被设置为假。如果有线程等待一个 event 对象，而这个 event 对象的标志为假，那么这个线程将会被一直阻塞直至该标志为真。一个线程如果将一个 event 对象的信号标志设置为真，它将唤醒所有等待这个 event 对象的线程。如果一个线程等待一个已经被设置为真的 event 对象，那么它将忽略这个事件，继续执行。
下边的代码展示了如何使用 Event 来协调线程的启动：

In [ ]:
f
r
o
m
 
t
h
r
e
a
d
i
n
g
 
i
m
p
o
r
t
 
T
h
r
e
a
d
,
 
E
v
e
n
t


i
m
p
o
r
t
 
t
i
m
e




#
 
C
o
d
e
 
t
o
 
e
x
e
c
u
t
e
 
i
n
 
a
n
 
i
n
d
e
p
e
n
d
e
n
t
 
t
h
r
e
a
d


d
e
f
 
c
o
u
n
t
d
o
w
n
(
n
,
 
s
t
a
r
t
e
d
_
e
v
t
)
:


 
 
 
 
p
r
i
n
t
(
'
c
o
u
n
t
d
o
w
n
 
s
t
a
r
t
i
n
g
'
)


 
 
 
 
s
t
a
r
t
e
d
_
e
v
t
.
s
e
t
(
)


 
 
 
 
w
h
i
l
e
 
n
 
>
 
0
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
T
-
m
i
n
u
s
'
,
 
n
)


 
 
 
 
 
 
 
 
n
 
-
=
 
1


 
 
 
 
 
 
 
 
t
i
m
e
.
s
l
e
e
p
(
5
)




#
 
C
r
e
a
t
e
 
t
h
e
 
e
v
e
n
t
 
o
b
j
e
c
t
 
t
h
a
t
 
w
i
l
l
 
b
e
 
u
s
e
d
 
t
o
 
s
i
g
n
a
l
 
s
t
a
r
t
u
p


s
t
a
r
t
e
d
_
e
v
t
 
=
 
E
v
e
n
t
(
)




#
 
L
a
u
n
c
h
 
t
h
e
 
t
h
r
e
a
d
 
a
n
d
 
p
a
s
s
 
t
h
e
 
s
t
a
r
t
u
p
 
e
v
e
n
t


p
r
i
n
t
(
'
L
a
u
n
c
h
i
n
g
 
c
o
u
n
t
d
o
w
n
'
)


t
 
=
 
T
h
r
e
a
d
(
t
a
r
g
e
t
=
c
o
u
n
t
d
o
w
n
,
 
a
r
g
s
=
(
1
0
,
s
t
a
r
t
e
d
_
e
v
t
)
)


t
.
s
t
a
r
t
(
)




#
 
W
a
i
t
 
f
o
r
 
t
h
e
 
t
h
r
e
a
d
 
t
o
 
s
t
a
r
t


s
t
a
r
t
e
d
_
e
v
t
.
w
a
i
t
(
)


p
r
i
n
t
(
'
c
o
u
n
t
d
o
w
n
 
i
s
 
r
u
n
n
i
n
g
'
)



当你执行这段代码，“countdown is running” 总是显示在 “countdown starting” 之后显示。这是由于使用 event 来协调线程，使得主线程要等到 countdown() 函数输出启动信息后，才能继续执行。

### 讨论


event 对象最好单次使用，就是说，你创建一个 event 对象，让某个线程等待这个对象，一旦这个对象被设置为真，你就应该丢弃它。尽管可以通过 clear() 方法来重置 event 对象，但是很难确保安全地清理 event 对象并对它重新赋值。很可能会发生错过事件、死锁或者其他问题（特别是，你无法保证重置 event 对象的代码会在线程再次等待这个 event 对象之前执行）。如果一个线程需要不停地重复使用 event 对象，你最好使用 Condition 对象来代替。下面的代码使用 Condition 对象实现了一个周期定时器，每当定时器超时的时候，其他线程都可以监测到：

In [ ]:
i
m
p
o
r
t
 
t
h
r
e
a
d
i
n
g


i
m
p
o
r
t
 
t
i
m
e




c
l
a
s
s
 
P
e
r
i
o
d
i
c
T
i
m
e
r
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
i
n
t
e
r
v
a
l
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
i
n
t
e
r
v
a
l
 
=
 
i
n
t
e
r
v
a
l


 
 
 
 
 
 
 
 
s
e
l
f
.
_
f
l
a
g
 
=
 
0


 
 
 
 
 
 
 
 
s
e
l
f
.
_
c
v
 
=
 
t
h
r
e
a
d
i
n
g
.
C
o
n
d
i
t
i
o
n
(
)




 
 
 
 
d
e
f
 
s
t
a
r
t
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
t
 
=
 
t
h
r
e
a
d
i
n
g
.
T
h
r
e
a
d
(
t
a
r
g
e
t
=
s
e
l
f
.
r
u
n
)


 
 
 
 
 
 
 
 
t
.
d
a
e
m
o
n
 
=
 
T
r
u
e




 
 
 
 
 
 
 
 
t
.
s
t
a
r
t
(
)




 
 
 
 
d
e
f
 
r
u
n
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
R
u
n
 
t
h
e
 
t
i
m
e
r
 
a
n
d
 
n
o
t
i
f
y
 
w
a
i
t
i
n
g
 
t
h
r
e
a
d
s
 
a
f
t
e
r
 
e
a
c
h
 
i
n
t
e
r
v
a
l


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
 
 
 
 
t
i
m
e
.
s
l
e
e
p
(
s
e
l
f
.
_
i
n
t
e
r
v
a
l
)


 
 
 
 
 
 
 
 
 
 
 
 
w
i
t
h
 
s
e
l
f
.
_
c
v
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
f
l
a
g
 
^
=
 
1


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
c
v
.
n
o
t
i
f
y
_
a
l
l
(
)




 
 
 
 
d
e
f
 
w
a
i
t
_
f
o
r
_
t
i
c
k
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
W
a
i
t
 
f
o
r
 
t
h
e
 
n
e
x
t
 
t
i
c
k
 
o
f
 
t
h
e
 
t
i
m
e
r


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
w
i
t
h
 
s
e
l
f
.
_
c
v
:


 
 
 
 
 
 
 
 
 
 
 
 
l
a
s
t
_
f
l
a
g
 
=
 
s
e
l
f
.
_
f
l
a
g


 
 
 
 
 
 
 
 
 
 
 
 
w
h
i
l
e
 
l
a
s
t
_
f
l
a
g
 
=
=
 
s
e
l
f
.
_
f
l
a
g
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
c
v
.
w
a
i
t
(
)




#
 
E
x
a
m
p
l
e
 
u
s
e
 
o
f
 
t
h
e
 
t
i
m
e
r


p
t
i
m
e
r
 
=
 
P
e
r
i
o
d
i
c
T
i
m
e
r
(
5
)


p
t
i
m
e
r
.
s
t
a
r
t
(
)




#
 
T
w
o
 
t
h
r
e
a
d
s
 
t
h
a
t
 
s
y
n
c
h
r
o
n
i
z
e
 
o
n
 
t
h
e
 
t
i
m
e
r


d
e
f
 
c
o
u
n
t
d
o
w
n
(
n
t
i
c
k
s
)
:


 
 
 
 
w
h
i
l
e
 
n
t
i
c
k
s
 
>
 
0
:


 
 
 
 
 
 
 
 
p
t
i
m
e
r
.
w
a
i
t
_
f
o
r
_
t
i
c
k
(
)


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
T
-
m
i
n
u
s
'
,
 
n
t
i
c
k
s
)


 
 
 
 
 
 
 
 
n
t
i
c
k
s
 
-
=
 
1




d
e
f
 
c
o
u
n
t
u
p
(
l
a
s
t
)
:


 
 
 
 
n
 
=
 
0


 
 
 
 
w
h
i
l
e
 
n
 
<
 
l
a
s
t
:


 
 
 
 
 
 
 
 
p
t
i
m
e
r
.
w
a
i
t
_
f
o
r
_
t
i
c
k
(
)


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
C
o
u
n
t
i
n
g
'
,
 
n
)


 
 
 
 
 
 
 
 
n
 
+
=
 
1




t
h
r
e
a
d
i
n
g
.
T
h
r
e
a
d
(
t
a
r
g
e
t
=
c
o
u
n
t
d
o
w
n
,
 
a
r
g
s
=
(
1
0
,
)
)
.
s
t
a
r
t
(
)


t
h
r
e
a
d
i
n
g
.
T
h
r
e
a
d
(
t
a
r
g
e
t
=
c
o
u
n
t
u
p
,
 
a
r
g
s
=
(
5
,
)
)
.
s
t
a
r
t
(
)



event对象的一个重要特点是当它被设置为真时会唤醒所有等待它的线程。如果你只想唤醒单个线程，最好是使用信号量或者 Condition 对象来替代。考虑一下这段使用信号量实现的代码：

In [ ]:
#
 
W
o
r
k
e
r
 
t
h
r
e
a
d


d
e
f
 
w
o
r
k
e
r
(
n
,
 
s
e
m
a
)
:


 
 
 
 
#
 
W
a
i
t
 
t
o
 
b
e
 
s
i
g
n
a
l
e
d


 
 
 
 
s
e
m
a
.
a
c
q
u
i
r
e
(
)




 
 
 
 
#
 
D
o
 
s
o
m
e
 
w
o
r
k


 
 
 
 
p
r
i
n
t
(
'
W
o
r
k
i
n
g
'
,
 
n
)




#
 
C
r
e
a
t
e
 
s
o
m
e
 
t
h
r
e
a
d
s


s
e
m
a
 
=
 
t
h
r
e
a
d
i
n
g
.
S
e
m
a
p
h
o
r
e
(
0
)


n
w
o
r
k
e
r
s
 
=
 
1
0


f
o
r
 
n
 
i
n
 
r
a
n
g
e
(
n
w
o
r
k
e
r
s
)
:


 
 
 
 
t
 
=
 
t
h
r
e
a
d
i
n
g
.
T
h
r
e
a
d
(
t
a
r
g
e
t
=
w
o
r
k
e
r
,
 
a
r
g
s
=
(
n
,
 
s
e
m
a
,
)
)


 
 
 
 
t
.
s
t
a
r
t
(
)



运行上边的代码将会启动一个线程池，但是并没有什么事情发生。这是因为所有的线程都在等待获取信号量。每次信号量被释放，只有一个线程会被唤醒并执行，示例如下：

In [ ]:
sema.release()

In [ ]:
sema.release()

编写涉及到大量的线程间同步问题的代码会让你痛不欲生。比较合适的方式是使用队列来进行线程间通信或者每个把线程当作一个Actor，利用Actor模型来控制并发。下一节将会介绍到队列，而Actor模型将在12.10节介绍。

## 12.3 线程间通信


### 问题


你的程序中有多个线程，你需要在这些线程之间安全地交换信息或数据

### 解决方案


从一个线程向另一个线程发送数据最安全的方式可能就是使用 queue 库中的队列了。创建一个被多个线程共享的 Queue 对象，这些线程通过使用 put() 和 get() 操作来向队列中添加或者删除元素。
例如：

In [ ]:
f
r
o
m
 
q
u
e
u
e
 
i
m
p
o
r
t
 
Q
u
e
u
e


f
r
o
m
 
t
h
r
e
a
d
i
n
g
 
i
m
p
o
r
t
 
T
h
r
e
a
d




#
 
A
 
t
h
r
e
a
d
 
t
h
a
t
 
p
r
o
d
u
c
e
s
 
d
a
t
a


d
e
f
 
p
r
o
d
u
c
e
r
(
o
u
t
_
q
)
:


 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
#
 
P
r
o
d
u
c
e
 
s
o
m
e
 
d
a
t
a


 
 
 
 
 
 
 
 
.
.
.


 
 
 
 
 
 
 
 
o
u
t
_
q
.
p
u
t
(
d
a
t
a
)




#
 
A
 
t
h
r
e
a
d
 
t
h
a
t
 
c
o
n
s
u
m
e
s
 
d
a
t
a


d
e
f
 
c
o
n
s
u
m
e
r
(
i
n
_
q
)
:


 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


#
 
G
e
t
 
s
o
m
e
 
d
a
t
a


 
 
 
 
 
 
 
 
d
a
t
a
 
=
 
i
n
_
q
.
g
e
t
(
)


 
 
 
 
 
 
 
 
#
 
P
r
o
c
e
s
s
 
t
h
e
 
d
a
t
a


 
 
 
 
 
 
 
 
.
.
.




#
 
C
r
e
a
t
e
 
t
h
e
 
s
h
a
r
e
d
 
q
u
e
u
e
 
a
n
d
 
l
a
u
n
c
h
 
b
o
t
h
 
t
h
r
e
a
d
s


q
 
=
 
Q
u
e
u
e
(
)


t
1
 
=
 
T
h
r
e
a
d
(
t
a
r
g
e
t
=
c
o
n
s
u
m
e
r
,
 
a
r
g
s
=
(
q
,
)
)


t
2
 
=
 
T
h
r
e
a
d
(
t
a
r
g
e
t
=
p
r
o
d
u
c
e
r
,
 
a
r
g
s
=
(
q
,
)
)


t
1
.
s
t
a
r
t
(
)


t
2
.
s
t
a
r
t
(
)



Queue 对象已经包含了必要的锁，所以你可以通过它在多个线程间多安全地共享数据。
当使用队列时，协调生产者和消费者的关闭问题可能会有一些麻烦。一个通用的解决方法是在队列中放置一个特殊的值，当消费者读到这个值的时候，终止执行。例如：

In [ ]:
f
r
o
m
 
q
u
e
u
e
 
i
m
p
o
r
t
 
Q
u
e
u
e


f
r
o
m
 
t
h
r
e
a
d
i
n
g
 
i
m
p
o
r
t
 
T
h
r
e
a
d




#
 
O
b
j
e
c
t
 
t
h
a
t
 
s
i
g
n
a
l
s
 
s
h
u
t
d
o
w
n


_
s
e
n
t
i
n
e
l
 
=
 
o
b
j
e
c
t
(
)




#
 
A
 
t
h
r
e
a
d
 
t
h
a
t
 
p
r
o
d
u
c
e
s
 
d
a
t
a


d
e
f
 
p
r
o
d
u
c
e
r
(
o
u
t
_
q
)
:


 
 
 
 
w
h
i
l
e
 
r
u
n
n
i
n
g
:


 
 
 
 
 
 
 
 
#
 
P
r
o
d
u
c
e
 
s
o
m
e
 
d
a
t
a


 
 
 
 
 
 
 
 
.
.
.


 
 
 
 
 
 
 
 
o
u
t
_
q
.
p
u
t
(
d
a
t
a
)




 
 
 
 
#
 
P
u
t
 
t
h
e
 
s
e
n
t
i
n
e
l
 
o
n
 
t
h
e
 
q
u
e
u
e
 
t
o
 
i
n
d
i
c
a
t
e
 
c
o
m
p
l
e
t
i
o
n


 
 
 
 
o
u
t
_
q
.
p
u
t
(
_
s
e
n
t
i
n
e
l
)




#
 
A
 
t
h
r
e
a
d
 
t
h
a
t
 
c
o
n
s
u
m
e
s
 
d
a
t
a


d
e
f
 
c
o
n
s
u
m
e
r
(
i
n
_
q
)
:


 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
#
 
G
e
t
 
s
o
m
e
 
d
a
t
a


 
 
 
 
 
 
 
 
d
a
t
a
 
=
 
i
n
_
q
.
g
e
t
(
)




 
 
 
 
 
 
 
 
#
 
C
h
e
c
k
 
f
o
r
 
t
e
r
m
i
n
a
t
i
o
n


 
 
 
 
 
 
 
 
i
f
 
d
a
t
a
 
i
s
 
_
s
e
n
t
i
n
e
l
:


 
 
 
 
 
 
 
 
 
 
 
 
i
n
_
q
.
p
u
t
(
_
s
e
n
t
i
n
e
l
)


 
 
 
 
 
 
 
 
 
 
 
 
b
r
e
a
k




 
 
 
 
 
 
 
 
#
 
P
r
o
c
e
s
s
 
t
h
e
 
d
a
t
a


 
 
 
 
 
 
 
 
.
.
.



本例中有一个特殊的地方：消费者在读到这个特殊值之后立即又把它放回到队列中，将之传递下去。这样，所有监听这个队列的消费者线程就可以全部关闭了。
尽管队列是最常见的线程间通信机制，但是仍然可以自己通过创建自己的数据结构并添加所需的锁和同步机制来实现线程间通信。最常见的方法是使用 Condition 变量来包装你的数据结构。下边这个例子演示了如何创建一个线程安全的优先级队列，如同1.5节中介绍的那样。

In [ ]:
i
m
p
o
r
t
 
h
e
a
p
q


i
m
p
o
r
t
 
t
h
r
e
a
d
i
n
g




c
l
a
s
s
 
P
r
i
o
r
i
t
y
Q
u
e
u
e
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
q
u
e
u
e
 
=
 
[
]


 
 
 
 
 
 
 
 
s
e
l
f
.
_
c
o
u
n
t
 
=
 
0


 
 
 
 
 
 
 
 
s
e
l
f
.
_
c
v
 
=
 
t
h
r
e
a
d
i
n
g
.
C
o
n
d
i
t
i
o
n
(
)


 
 
 
 
d
e
f
 
p
u
t
(
s
e
l
f
,
 
i
t
e
m
,
 
p
r
i
o
r
i
t
y
)
:


 
 
 
 
 
 
 
 
w
i
t
h
 
s
e
l
f
.
_
c
v
:


 
 
 
 
 
 
 
 
 
 
 
 
h
e
a
p
q
.
h
e
a
p
p
u
s
h
(
s
e
l
f
.
_
q
u
e
u
e
,
 
(
-
p
r
i
o
r
i
t
y
,
 
s
e
l
f
.
_
c
o
u
n
t
,
 
i
t
e
m
)
)


 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
c
o
u
n
t
 
+
=
 
1


 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
c
v
.
n
o
t
i
f
y
(
)




 
 
 
 
d
e
f
 
g
e
t
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
w
i
t
h
 
s
e
l
f
.
_
c
v
:


 
 
 
 
 
 
 
 
 
 
 
 
w
h
i
l
e
 
l
e
n
(
s
e
l
f
.
_
q
u
e
u
e
)
 
=
=
 
0
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
c
v
.
w
a
i
t
(
)


 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
h
e
a
p
q
.
h
e
a
p
p
o
p
(
s
e
l
f
.
_
q
u
e
u
e
)
[
-
1
]



使用队列来进行线程间通信是一个单向、不确定的过程。通常情况下，你没有办法知道接收数据的线程是什么时候接收到的数据并开始工作的。不过队列对象提供一些基本完成的特性，比如下边这个例子中的 task_done() 和 join() ：

In [ ]:
f
r
o
m
 
q
u
e
u
e
 
i
m
p
o
r
t
 
Q
u
e
u
e


f
r
o
m
 
t
h
r
e
a
d
i
n
g
 
i
m
p
o
r
t
 
T
h
r
e
a
d




#
 
A
 
t
h
r
e
a
d
 
t
h
a
t
 
p
r
o
d
u
c
e
s
 
d
a
t
a


d
e
f
 
p
r
o
d
u
c
e
r
(
o
u
t
_
q
)
:


 
 
 
 
w
h
i
l
e
 
r
u
n
n
i
n
g
:


 
 
 
 
 
 
 
 
#
 
P
r
o
d
u
c
e
 
s
o
m
e
 
d
a
t
a


 
 
 
 
 
 
 
 
.
.
.


 
 
 
 
 
 
 
 
o
u
t
_
q
.
p
u
t
(
d
a
t
a
)




#
 
A
 
t
h
r
e
a
d
 
t
h
a
t
 
c
o
n
s
u
m
e
s
 
d
a
t
a


d
e
f
 
c
o
n
s
u
m
e
r
(
i
n
_
q
)
:


 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
#
 
G
e
t
 
s
o
m
e
 
d
a
t
a


 
 
 
 
 
 
 
 
d
a
t
a
 
=
 
i
n
_
q
.
g
e
t
(
)




 
 
 
 
 
 
 
 
#
 
P
r
o
c
e
s
s
 
t
h
e
 
d
a
t
a


 
 
 
 
 
 
 
 
.
.
.


 
 
 
 
 
 
 
 
#
 
I
n
d
i
c
a
t
e
 
c
o
m
p
l
e
t
i
o
n


 
 
 
 
 
 
 
 
i
n
_
q
.
t
a
s
k
_
d
o
n
e
(
)




#
 
C
r
e
a
t
e
 
t
h
e
 
s
h
a
r
e
d
 
q
u
e
u
e
 
a
n
d
 
l
a
u
n
c
h
 
b
o
t
h
 
t
h
r
e
a
d
s


q
 
=
 
Q
u
e
u
e
(
)


t
1
 
=
 
T
h
r
e
a
d
(
t
a
r
g
e
t
=
c
o
n
s
u
m
e
r
,
 
a
r
g
s
=
(
q
,
)
)


t
2
 
=
 
T
h
r
e
a
d
(
t
a
r
g
e
t
=
p
r
o
d
u
c
e
r
,
 
a
r
g
s
=
(
q
,
)
)


t
1
.
s
t
a
r
t
(
)


t
2
.
s
t
a
r
t
(
)




#
 
W
a
i
t
 
f
o
r
 
a
l
l
 
p
r
o
d
u
c
e
d
 
i
t
e
m
s
 
t
o
 
b
e
 
c
o
n
s
u
m
e
d


q
.
j
o
i
n
(
)



如果一个线程需要在一个“消费者”线程处理完特定的数据项时立即得到通知，你可以把要发送的数据和一个 Event 放到一起使用，这样“生产者”就可以通过这个Event对象来监测处理的过程了。示例如下：

In [ ]:
f
r
o
m
 
q
u
e
u
e
 
i
m
p
o
r
t
 
Q
u
e
u
e


f
r
o
m
 
t
h
r
e
a
d
i
n
g
 
i
m
p
o
r
t
 
T
h
r
e
a
d
,
 
E
v
e
n
t




#
 
A
 
t
h
r
e
a
d
 
t
h
a
t
 
p
r
o
d
u
c
e
s
 
d
a
t
a


d
e
f
 
p
r
o
d
u
c
e
r
(
o
u
t
_
q
)
:


 
 
 
 
w
h
i
l
e
 
r
u
n
n
i
n
g
:


 
 
 
 
 
 
 
 
#
 
P
r
o
d
u
c
e
 
s
o
m
e
 
d
a
t
a


 
 
 
 
 
 
 
 
.
.
.


 
 
 
 
 
 
 
 
#
 
M
a
k
e
 
a
n
 
(
d
a
t
a
,
 
e
v
e
n
t
)
 
p
a
i
r
 
a
n
d
 
h
a
n
d
 
i
t
 
t
o
 
t
h
e
 
c
o
n
s
u
m
e
r


 
 
 
 
 
 
 
 
e
v
t
 
=
 
E
v
e
n
t
(
)


 
 
 
 
 
 
 
 
o
u
t
_
q
.
p
u
t
(
(
d
a
t
a
,
 
e
v
t
)
)


 
 
 
 
 
 
 
 
.
.
.


 
 
 
 
 
 
 
 
#
 
W
a
i
t
 
f
o
r
 
t
h
e
 
c
o
n
s
u
m
e
r
 
t
o
 
p
r
o
c
e
s
s
 
t
h
e
 
i
t
e
m


 
 
 
 
 
 
 
 
e
v
t
.
w
a
i
t
(
)




#
 
A
 
t
h
r
e
a
d
 
t
h
a
t
 
c
o
n
s
u
m
e
s
 
d
a
t
a


d
e
f
 
c
o
n
s
u
m
e
r
(
i
n
_
q
)
:


 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
#
 
G
e
t
 
s
o
m
e
 
d
a
t
a


 
 
 
 
 
 
 
 
d
a
t
a
,
 
e
v
t
 
=
 
i
n
_
q
.
g
e
t
(
)


 
 
 
 
 
 
 
 
#
 
P
r
o
c
e
s
s
 
t
h
e
 
d
a
t
a


 
 
 
 
 
 
 
 
.
.
.


 
 
 
 
 
 
 
 
#
 
I
n
d
i
c
a
t
e
 
c
o
m
p
l
e
t
i
o
n


 
 
 
 
 
 
 
 
e
v
t
.
s
e
t
(
)



### 讨论


基于简单队列编写多线程程序在多数情况下是一个比较明智的选择。从线程安全队列的底层实现来看，你无需在你的代码中使用锁和其他底层的同步机制，这些只会把你的程序弄得乱七八糟。此外，使用队列这种基于消息的通信机制可以被扩展到更大的应用范畴，比如，你可以把你的程序放入多个进程甚至是分布式系统而无需改变底层的队列结构。
使用线程队列有一个要注意的问题是，向队列中添加数据项时并不会复制此数据项，线程间通信实际上是在线程间传递对象引用。如果你担心对象的共享状态，那你最好只传递不可修改的数据结构（如：整型、字符串或者元组）或者一个对象的深拷贝。例如：

In [ ]:
f
r
o
m
 
q
u
e
u
e
 
i
m
p
o
r
t
 
Q
u
e
u
e


f
r
o
m
 
t
h
r
e
a
d
i
n
g
 
i
m
p
o
r
t
 
T
h
r
e
a
d


i
m
p
o
r
t
 
c
o
p
y




#
 
A
 
t
h
r
e
a
d
 
t
h
a
t
 
p
r
o
d
u
c
e
s
 
d
a
t
a


d
e
f
 
p
r
o
d
u
c
e
r
(
o
u
t
_
q
)
:


 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
#
 
P
r
o
d
u
c
e
 
s
o
m
e
 
d
a
t
a


 
 
 
 
 
 
 
 
.
.
.


 
 
 
 
 
 
 
 
o
u
t
_
q
.
p
u
t
(
c
o
p
y
.
d
e
e
p
c
o
p
y
(
d
a
t
a
)
)




#
 
A
 
t
h
r
e
a
d
 
t
h
a
t
 
c
o
n
s
u
m
e
s
 
d
a
t
a


d
e
f
 
c
o
n
s
u
m
e
r
(
i
n
_
q
)
:


 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
#
 
G
e
t
 
s
o
m
e
 
d
a
t
a


 
 
 
 
 
 
 
 
d
a
t
a
 
=
 
i
n
_
q
.
g
e
t
(
)


 
 
 
 
 
 
 
 
#
 
P
r
o
c
e
s
s
 
t
h
e
 
d
a
t
a


 
 
 
 
 
 
 
 
.
.
.



Queue 对象提供一些在当前上下文很有用的附加特性。比如在创建 Queue 对象时提供可选的 size 参数来限制可以添加到队列中的元素数量。对于“生产者”与“消费者”速度有差异的情况，为队列中的元素数量添加上限是有意义的。比如，一个“生产者”产生项目的速度比“消费者” “消费”的速度快，那么使用固定大小的队列就可以在队列已满的时候阻塞队列，以免未预期的连锁效应扩散整个程序造成死锁或者程序运行失常。在通信的线程之间进行“流量控制”是一个看起来容易实现起来困难的问题。如果你发现自己曾经试图通过摆弄队列大小来解决一个问题，这也许就标志着你的程序可能存在脆弱设计或者固有的可伸缩问题。
get() 和 put() 方法都支持非阻塞方式和设定超时，例如：

In [ ]:
i
m
p
o
r
t
 
q
u
e
u
e


q
 
=
 
q
u
e
u
e
.
Q
u
e
u
e
(
)




t
r
y
:


 
 
 
 
d
a
t
a
 
=
 
q
.
g
e
t
(
b
l
o
c
k
=
F
a
l
s
e
)


e
x
c
e
p
t
 
q
u
e
u
e
.
E
m
p
t
y
:


 
 
 
 
.
.
.




t
r
y
:


 
 
 
 
q
.
p
u
t
(
i
t
e
m
,
 
b
l
o
c
k
=
F
a
l
s
e
)


e
x
c
e
p
t
 
q
u
e
u
e
.
F
u
l
l
:


 
 
 
 
.
.
.




t
r
y
:


 
 
 
 
d
a
t
a
 
=
 
q
.
g
e
t
(
t
i
m
e
o
u
t
=
5
.
0
)


e
x
c
e
p
t
 
q
u
e
u
e
.
E
m
p
t
y
:


 
 
 
 
.
.
.



这些操作都可以用来避免当执行某些特定队列操作时发生无限阻塞的情况，比如，一个非阻塞的 put() 方法和一个固定大小的队列一起使用，这样当队列已满时就可以执行不同的代码。比如输出一条日志信息并丢弃。

In [ ]:
d
e
f
 
p
r
o
d
u
c
e
r
(
q
)
:


 
 
 
 
.
.
.


 
 
 
 
t
r
y
:


 
 
 
 
 
 
 
 
q
.
p
u
t
(
i
t
e
m
,
 
b
l
o
c
k
=
F
a
l
s
e
)


 
 
 
 
e
x
c
e
p
t
 
q
u
e
u
e
.
F
u
l
l
:


 
 
 
 
 
 
 
 
l
o
g
.
w
a
r
n
i
n
g
(
'
q
u
e
u
e
d
 
i
t
e
m
 
%
r
 
d
i
s
c
a
r
d
e
d
!
'
,
 
i
t
e
m
)



如果你试图让消费者线程在执行像 q.get() 这样的操作时，超时自动终止以便检查终止标志，你应该使用 q.get() 的可选参数 timeout ，如下：

In [ ]:
_
r
u
n
n
i
n
g
 
=
 
T
r
u
e




d
e
f
 
c
o
n
s
u
m
e
r
(
q
)
:


 
 
 
 
w
h
i
l
e
 
_
r
u
n
n
i
n
g
:


 
 
 
 
 
 
 
 
t
r
y
:


 
 
 
 
 
 
 
 
 
 
 
 
i
t
e
m
 
=
 
q
.
g
e
t
(
t
i
m
e
o
u
t
=
5
.
0
)


 
 
 
 
 
 
 
 
 
 
 
 
#
 
P
r
o
c
e
s
s
 
i
t
e
m


 
 
 
 
 
 
 
 
 
 
 
 
.
.
.


 
 
 
 
 
 
 
 
e
x
c
e
p
t
 
q
u
e
u
e
.
E
m
p
t
y
:


 
 
 
 
 
 
 
 
 
 
 
 
p
a
s
s



最后，有 q.qsize() ， q.full() ， q.empty() 等实用方法可以获取一个队列的当前大小和状态。但要注意，这些方法都不是线程安全的。可能你对一个队列使用 empty() 判断出这个队列为空，但同时另外一个线程可能已经向这个队列中插入一个数据项。所以，你最好不要在你的代码中使用这些方法。

## 12.4 给关键部分加锁


### 问题


你需要对多线程程序中的临界区加锁以避免竞争条件。

### 解决方案


要在多线程程序中安全使用可变对象，你需要使用 threading 库中的 Lock 对象，就像下边这个例子这样：

In [ ]:
i
m
p
o
r
t
 
t
h
r
e
a
d
i
n
g




c
l
a
s
s
 
S
h
a
r
e
d
C
o
u
n
t
e
r
:


 
 
 
 
'
'
'


 
 
 
 
A
 
c
o
u
n
t
e
r
 
o
b
j
e
c
t
 
t
h
a
t
 
c
a
n
 
b
e
 
s
h
a
r
e
d
 
b
y
 
m
u
l
t
i
p
l
e
 
t
h
r
e
a
d
s
.


 
 
 
 
'
'
'


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
i
n
i
t
i
a
l
_
v
a
l
u
e
 
=
 
0
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
v
a
l
u
e
 
=
 
i
n
i
t
i
a
l
_
v
a
l
u
e


 
 
 
 
 
 
 
 
s
e
l
f
.
_
v
a
l
u
e
_
l
o
c
k
 
=
 
t
h
r
e
a
d
i
n
g
.
L
o
c
k
(
)




 
 
 
 
d
e
f
 
i
n
c
r
(
s
e
l
f
,
d
e
l
t
a
=
1
)
:


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
I
n
c
r
e
m
e
n
t
 
t
h
e
 
c
o
u
n
t
e
r
 
w
i
t
h
 
l
o
c
k
i
n
g


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
w
i
t
h
 
s
e
l
f
.
_
v
a
l
u
e
_
l
o
c
k
:


 
 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
v
a
l
u
e
 
+
=
 
d
e
l
t
a




 
 
 
 
d
e
f
 
d
e
c
r
(
s
e
l
f
,
d
e
l
t
a
=
1
)
:


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
D
e
c
r
e
m
e
n
t
 
t
h
e
 
c
o
u
n
t
e
r
 
w
i
t
h
 
l
o
c
k
i
n
g


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
w
i
t
h
 
s
e
l
f
.
_
v
a
l
u
e
_
l
o
c
k
:


 
 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
v
a
l
u
e
 
-
=
 
d
e
l
t
a



Lock 对象和 with 语句块一起使用可以保证互斥执行，就是每次只有一个线程可以执行 with 语句包含的代码块。with 语句会在这个代码块执行前自动获取锁，在执行结束后自动释放锁。

### 讨论


线程调度本质上是不确定的，因此，在多线程程序中错误地使用锁机制可能会导致随机数据损坏或者其他的异常行为，我们称之为竞争条件。为了避免竞争条件，最好只在临界区（对临界资源进行操作的那部分代码）使用锁。
在一些“老的” Python 代码中，显式获取和释放锁是很常见的。下边是一个上一个例子的变种：

In [ ]:
i
m
p
o
r
t
 
t
h
r
e
a
d
i
n
g




c
l
a
s
s
 
S
h
a
r
e
d
C
o
u
n
t
e
r
:


 
 
 
 
'
'
'


 
 
 
 
A
 
c
o
u
n
t
e
r
 
o
b
j
e
c
t
 
t
h
a
t
 
c
a
n
 
b
e
 
s
h
a
r
e
d
 
b
y
 
m
u
l
t
i
p
l
e
 
t
h
r
e
a
d
s
.


 
 
 
 
'
'
'


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
i
n
i
t
i
a
l
_
v
a
l
u
e
 
=
 
0
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
v
a
l
u
e
 
=
 
i
n
i
t
i
a
l
_
v
a
l
u
e


 
 
 
 
 
 
 
 
s
e
l
f
.
_
v
a
l
u
e
_
l
o
c
k
 
=
 
t
h
r
e
a
d
i
n
g
.
L
o
c
k
(
)




 
 
 
 
d
e
f
 
i
n
c
r
(
s
e
l
f
,
d
e
l
t
a
=
1
)
:


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
I
n
c
r
e
m
e
n
t
 
t
h
e
 
c
o
u
n
t
e
r
 
w
i
t
h
 
l
o
c
k
i
n
g


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
s
e
l
f
.
_
v
a
l
u
e
_
l
o
c
k
.
a
c
q
u
i
r
e
(
)


 
 
 
 
 
 
 
 
s
e
l
f
.
_
v
a
l
u
e
 
+
=
 
d
e
l
t
a


 
 
 
 
 
 
 
 
s
e
l
f
.
_
v
a
l
u
e
_
l
o
c
k
.
r
e
l
e
a
s
e
(
)




 
 
 
 
d
e
f
 
d
e
c
r
(
s
e
l
f
,
d
e
l
t
a
=
1
)
:


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
D
e
c
r
e
m
e
n
t
 
t
h
e
 
c
o
u
n
t
e
r
 
w
i
t
h
 
l
o
c
k
i
n
g


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
s
e
l
f
.
_
v
a
l
u
e
_
l
o
c
k
.
a
c
q
u
i
r
e
(
)


 
 
 
 
 
 
 
 
s
e
l
f
.
_
v
a
l
u
e
 
-
=
 
d
e
l
t
a


 
 
 
 
 
 
 
 
s
e
l
f
.
_
v
a
l
u
e
_
l
o
c
k
.
r
e
l
e
a
s
e
(
)



相比于这种显式调用的方法，with 语句更加优雅，也更不容易出错，特别是程序员可能会忘记调用 release() 方法或者程序在获得锁之后产生异常这两种情况（使用 with 语句可以保证在这两种情况下仍能正确释放锁）。
为了避免出现死锁的情况，使用锁机制的程序应该设定为每个线程一次只允许获取一个锁。如果不能这样做的话，你就需要更高级的死锁避免机制，我们将在12.5节介绍。
在 threading 库中还提供了其他的同步原语，比如 RLock 和 Semaphore 对象。但是根据以往经验，这些原语是用于一些特殊的情况，如果你只是需要简单地对可变对象进行锁定，那就不应该使用它们。一个 RLock （可重入锁）可以被同一个线程多次获取，主要用来实现基于监测对象模式的锁定和同步。在使用这种锁的情况下，当锁被持有时，只有一个线程可以使用完整的函数或者类中的方法。比如，你可以实现一个这样的 SharedCounter 类：

In [ ]:
i
m
p
o
r
t
 
t
h
r
e
a
d
i
n
g




c
l
a
s
s
 
S
h
a
r
e
d
C
o
u
n
t
e
r
:


 
 
 
 
'
'
'


 
 
 
 
A
 
c
o
u
n
t
e
r
 
o
b
j
e
c
t
 
t
h
a
t
 
c
a
n
 
b
e
 
s
h
a
r
e
d
 
b
y
 
m
u
l
t
i
p
l
e
 
t
h
r
e
a
d
s
.


 
 
 
 
'
'
'


 
 
 
 
_
l
o
c
k
 
=
 
t
h
r
e
a
d
i
n
g
.
R
L
o
c
k
(
)


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
i
n
i
t
i
a
l
_
v
a
l
u
e
 
=
 
0
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
v
a
l
u
e
 
=
 
i
n
i
t
i
a
l
_
v
a
l
u
e




 
 
 
 
d
e
f
 
i
n
c
r
(
s
e
l
f
,
d
e
l
t
a
=
1
)
:


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
I
n
c
r
e
m
e
n
t
 
t
h
e
 
c
o
u
n
t
e
r
 
w
i
t
h
 
l
o
c
k
i
n
g


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
w
i
t
h
 
S
h
a
r
e
d
C
o
u
n
t
e
r
.
_
l
o
c
k
:


 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
v
a
l
u
e
 
+
=
 
d
e
l
t
a




 
 
 
 
d
e
f
 
d
e
c
r
(
s
e
l
f
,
d
e
l
t
a
=
1
)
:


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
D
e
c
r
e
m
e
n
t
 
t
h
e
 
c
o
u
n
t
e
r
 
w
i
t
h
 
l
o
c
k
i
n
g


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
w
i
t
h
 
S
h
a
r
e
d
C
o
u
n
t
e
r
.
_
l
o
c
k
:


 
 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
i
n
c
r
(
-
d
e
l
t
a
)



在上边这个例子中，没有对每一个实例中的可变对象加锁，取而代之的是一个被所有实例共享的类级锁。这个锁用来同步类方法，具体来说就是，这个锁可以保证一次只有一个线程可以调用这个类方法。不过，与一个标准的锁不同的是，已经持有这个锁的方法在调用同样使用这个锁的方法时，无需再次获取锁。比如 decr 方法。
这种实现方式的一个特点是，无论这个类有多少个实例都只用一个锁。因此在需要大量使用计数器的情况下内存效率更高。不过这样做也有缺点，就是在程序中使用大量线程并频繁更新计数器时会有争用锁的问题。
信号量对象是一个建立在共享计数器基础上的同步原语。如果计数器不为0，with 语句将计数器减1，线程被允许执行。with 语句执行结束后，计数器加１。如果计数器为0，线程将被阻塞，直到其他线程结束将计数器加1。尽管你可以在程序中像标准锁一样使用信号量来做线程同步，但是这种方式并不被推荐，因为使用信号量为程序增加的复杂性会影响程序性能。相对于简单地作为锁使用，信号量更适用于那些需要在线程之间引入信号或者限制的程序。比如，你需要限制一段代码的并发访问量，你就可以像下面这样使用信号量完成：

In [ ]:
f
r
o
m
 
t
h
r
e
a
d
i
n
g
 
i
m
p
o
r
t
 
S
e
m
a
p
h
o
r
e


i
m
p
o
r
t
 
u
r
l
l
i
b
.
r
e
q
u
e
s
t




#
 
A
t
 
m
o
s
t
,
 
f
i
v
e
 
t
h
r
e
a
d
s
 
a
l
l
o
w
e
d
 
t
o
 
r
u
n
 
a
t
 
o
n
c
e


_
f
e
t
c
h
_
u
r
l
_
s
e
m
a
 
=
 
S
e
m
a
p
h
o
r
e
(
5
)




d
e
f
 
f
e
t
c
h
_
u
r
l
(
u
r
l
)
:


 
 
 
 
w
i
t
h
 
_
f
e
t
c
h
_
u
r
l
_
s
e
m
a
:


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
u
r
l
l
i
b
.
r
e
q
u
e
s
t
.
u
r
l
o
p
e
n
(
u
r
l
)



如果你对线程同步原语的底层理论和实现感兴趣，可以参考操作系统相关书籍，绝大多数都有提及。

## 12.5 防止死锁的加锁机制


### 问题


你正在写一个多线程程序，其中线程需要一次获取多个锁，此时如何避免死锁问题。

### 解决方案


在多线程程序中，死锁问题很大一部分是由于线程同时获取多个锁造成的。举个例子：一个线程获取了第一个锁，然后在获取第二个锁的
时候发生阻塞，那么这个线程就可能阻塞其他线程的执行，从而导致整个程序假死。
解决死锁问题的一种方案是为程序中的每一个锁分配一个唯一的id，然后只允许按照升序规则来使用多个锁，这个规则使用上下文管理器
是非常容易实现的，示例如下：

In [ ]:
i
m
p
o
r
t
 
t
h
r
e
a
d
i
n
g


f
r
o
m
 
c
o
n
t
e
x
t
l
i
b
 
i
m
p
o
r
t
 
c
o
n
t
e
x
t
m
a
n
a
g
e
r




#
 
T
h
r
e
a
d
-
l
o
c
a
l
 
s
t
a
t
e
 
t
o
 
s
t
o
r
e
d
 
i
n
f
o
r
m
a
t
i
o
n
 
o
n
 
l
o
c
k
s
 
a
l
r
e
a
d
y
 
a
c
q
u
i
r
e
d


_
l
o
c
a
l
 
=
 
t
h
r
e
a
d
i
n
g
.
l
o
c
a
l
(
)




@
c
o
n
t
e
x
t
m
a
n
a
g
e
r


d
e
f
 
a
c
q
u
i
r
e
(
*
l
o
c
k
s
)
:


 
 
 
 
#
 
S
o
r
t
 
l
o
c
k
s
 
b
y
 
o
b
j
e
c
t
 
i
d
e
n
t
i
f
i
e
r


 
 
 
 
l
o
c
k
s
 
=
 
s
o
r
t
e
d
(
l
o
c
k
s
,
 
k
e
y
=
l
a
m
b
d
a
 
x
:
 
i
d
(
x
)
)




 
 
 
 
#
 
M
a
k
e
 
s
u
r
e
 
l
o
c
k
 
o
r
d
e
r
 
o
f
 
p
r
e
v
i
o
u
s
l
y
 
a
c
q
u
i
r
e
d
 
l
o
c
k
s
 
i
s
 
n
o
t
 
v
i
o
l
a
t
e
d


 
 
 
 
a
c
q
u
i
r
e
d
 
=
 
g
e
t
a
t
t
r
(
_
l
o
c
a
l
,
'
a
c
q
u
i
r
e
d
'
,
[
]
)


 
 
 
 
i
f
 
a
c
q
u
i
r
e
d
 
a
n
d
 
m
a
x
(
i
d
(
l
o
c
k
)
 
f
o
r
 
l
o
c
k
 
i
n
 
a
c
q
u
i
r
e
d
)
 
>
=
 
i
d
(
l
o
c
k
s
[
0
]
)
:


 
 
 
 
 
 
 
 
r
a
i
s
e
 
R
u
n
t
i
m
e
E
r
r
o
r
(
'
L
o
c
k
 
O
r
d
e
r
 
V
i
o
l
a
t
i
o
n
'
)




 
 
 
 
#
 
A
c
q
u
i
r
e
 
a
l
l
 
o
f
 
t
h
e
 
l
o
c
k
s


 
 
 
 
a
c
q
u
i
r
e
d
.
e
x
t
e
n
d
(
l
o
c
k
s
)


 
 
 
 
_
l
o
c
a
l
.
a
c
q
u
i
r
e
d
 
=
 
a
c
q
u
i
r
e
d




 
 
 
 
t
r
y
:


 
 
 
 
 
 
 
 
f
o
r
 
l
o
c
k
 
i
n
 
l
o
c
k
s
:


 
 
 
 
 
 
 
 
 
 
 
 
l
o
c
k
.
a
c
q
u
i
r
e
(
)


 
 
 
 
 
 
 
 
y
i
e
l
d


 
 
 
 
f
i
n
a
l
l
y
:


 
 
 
 
 
 
 
 
#
 
R
e
l
e
a
s
e
 
l
o
c
k
s
 
i
n
 
r
e
v
e
r
s
e
 
o
r
d
e
r
 
o
f
 
a
c
q
u
i
s
i
t
i
o
n


 
 
 
 
 
 
 
 
f
o
r
 
l
o
c
k
 
i
n
 
r
e
v
e
r
s
e
d
(
l
o
c
k
s
)
:


 
 
 
 
 
 
 
 
 
 
 
 
l
o
c
k
.
r
e
l
e
a
s
e
(
)


 
 
 
 
 
 
 
 
d
e
l
 
a
c
q
u
i
r
e
d
[
-
l
e
n
(
l
o
c
k
s
)
:
]



如何使用这个上下文管理器呢？你可以按照正常途径创建一个锁对象，但不论是单个锁还是多个锁中都使用 acquire() 函数来申请锁，
示例如下：

In [ ]:
i
m
p
o
r
t
 
t
h
r
e
a
d
i
n
g


x
_
l
o
c
k
 
=
 
t
h
r
e
a
d
i
n
g
.
L
o
c
k
(
)


y
_
l
o
c
k
 
=
 
t
h
r
e
a
d
i
n
g
.
L
o
c
k
(
)




d
e
f
 
t
h
r
e
a
d
_
1
(
)
:


 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
w
i
t
h
 
a
c
q
u
i
r
e
(
x
_
l
o
c
k
,
 
y
_
l
o
c
k
)
:


 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
T
h
r
e
a
d
-
1
'
)




d
e
f
 
t
h
r
e
a
d
_
2
(
)
:


 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
w
i
t
h
 
a
c
q
u
i
r
e
(
y
_
l
o
c
k
,
 
x
_
l
o
c
k
)
:


 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
T
h
r
e
a
d
-
2
'
)




t
1
 
=
 
t
h
r
e
a
d
i
n
g
.
T
h
r
e
a
d
(
t
a
r
g
e
t
=
t
h
r
e
a
d
_
1
)


t
1
.
d
a
e
m
o
n
 
=
 
T
r
u
e


t
1
.
s
t
a
r
t
(
)




t
2
 
=
 
t
h
r
e
a
d
i
n
g
.
T
h
r
e
a
d
(
t
a
r
g
e
t
=
t
h
r
e
a
d
_
2
)


t
2
.
d
a
e
m
o
n
 
=
 
T
r
u
e


t
2
.
s
t
a
r
t
(
)



如果你执行这段代码，你会发现它即使在不同的函数中以不同的顺序获取锁也没有发生死锁。
其关键在于，在第一段代码中，我们对这些锁进行了排序。通过排序，使得不管用户以什么样的顺序来请求锁，这些锁都会按照固定的顺序被获取。
如果有多个 acquire() 操作被嵌套调用，可以通过线程本地存储（TLS）来检测潜在的死锁问题。
假设你的代码是这样写的：

In [ ]:
i
m
p
o
r
t
 
t
h
r
e
a
d
i
n
g


x
_
l
o
c
k
 
=
 
t
h
r
e
a
d
i
n
g
.
L
o
c
k
(
)


y
_
l
o
c
k
 
=
 
t
h
r
e
a
d
i
n
g
.
L
o
c
k
(
)




d
e
f
 
t
h
r
e
a
d
_
1
(
)
:




 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
w
i
t
h
 
a
c
q
u
i
r
e
(
x
_
l
o
c
k
)
:


 
 
 
 
 
 
 
 
 
 
 
 
w
i
t
h
 
a
c
q
u
i
r
e
(
y
_
l
o
c
k
)
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
T
h
r
e
a
d
-
1
'
)




d
e
f
 
t
h
r
e
a
d
_
2
(
)
:


 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
w
i
t
h
 
a
c
q
u
i
r
e
(
y
_
l
o
c
k
)
:


 
 
 
 
 
 
 
 
 
 
 
 
w
i
t
h
 
a
c
q
u
i
r
e
(
x
_
l
o
c
k
)
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
T
h
r
e
a
d
-
2
'
)




t
1
 
=
 
t
h
r
e
a
d
i
n
g
.
T
h
r
e
a
d
(
t
a
r
g
e
t
=
t
h
r
e
a
d
_
1
)


t
1
.
d
a
e
m
o
n
 
=
 
T
r
u
e


t
1
.
s
t
a
r
t
(
)




t
2
 
=
 
t
h
r
e
a
d
i
n
g
.
T
h
r
e
a
d
(
t
a
r
g
e
t
=
t
h
r
e
a
d
_
2
)


t
2
.
d
a
e
m
o
n
 
=
 
T
r
u
e


t
2
.
s
t
a
r
t
(
)



如果你运行这个版本的代码，必定会有一个线程发生崩溃，异常信息可能像这样：

发生崩溃的原因在于，每个线程都记录着自己已经获取到的锁。 acquire() 函数会检查之前已经获取的锁列表，
由于锁是按照升序排列获取的，所以函数会认为之前已获取的锁的id必定小于新申请到的锁，这时就会触发异常。

### 讨论


死锁是每一个多线程程序都会面临的一个问题（就像它是每一本操作系统课本的共同话题一样）。根据经验来讲，尽可能保证每一个
线程只能同时保持一个锁，这样程序就不会被死锁问题所困扰。一旦有线程同时申请多个锁，一切就不可预料了。

死锁的检测与恢复是一个几乎没有优雅的解决方案的扩展话题。一个比较常用的死锁检测与恢复的方案是引入看门狗计数器。当线程正常
运行的时候会每隔一段时间重置计数器，在没有发生死锁的情况下，一切都正常进行。一旦发生死锁，由于无法重置计数器导致定时器
超时，这时程序会通过重启自身恢复到正常状态。

避免死锁是另外一种解决死锁问题的方式，在进程获取锁的时候会严格按照对象id升序排列获取，经过数学证明，这样保证程序不会进入
死锁状态。证明就留给读者作为练习了。避免死锁的主要思想是，单纯地按照对象id递增的顺序加锁不会产生循环依赖，而循环依赖是
死锁的一个必要条件，从而避免程序进入死锁状态。

下面以一个关于线程死锁的经典问题：“哲学家就餐问题”，作为本节最后一个例子。题目是这样的：五位哲学家围坐在一张桌子前，每个人
面前有一碗饭和一只筷子。在这里每个哲学家可以看做是一个独立的线程，而每只筷子可以看做是一个锁。每个哲学家可以处在静坐、
思考、吃饭三种状态中的一个。需要注意的是，每个哲学家吃饭是需要两只筷子的，这样问题就来了：如果每个哲学家都拿起自己左边的筷子，
那么他们五个都只能拿着一只筷子坐在那儿，直到饿死。此时他们就进入了死锁状态。
下面是一个简单的使用死锁避免机制解决“哲学家就餐问题”的实现：

In [ ]:
i
m
p
o
r
t
 
t
h
r
e
a
d
i
n
g




#
 
T
h
e
 
p
h
i
l
o
s
o
p
h
e
r
 
t
h
r
e
a
d


d
e
f
 
p
h
i
l
o
s
o
p
h
e
r
(
l
e
f
t
,
 
r
i
g
h
t
)
:


 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
w
i
t
h
 
a
c
q
u
i
r
e
(
l
e
f
t
,
r
i
g
h
t
)
:


 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
t
h
r
e
a
d
i
n
g
.
c
u
r
r
e
n
t
T
h
r
e
a
d
(
)
,
 
'
e
a
t
i
n
g
'
)




#
 
T
h
e
 
c
h
o
p
s
t
i
c
k
s
 
(
r
e
p
r
e
s
e
n
t
e
d
 
b
y
 
l
o
c
k
s
)


N
S
T
I
C
K
S
 
=
 
5


c
h
o
p
s
t
i
c
k
s
 
=
 
[
t
h
r
e
a
d
i
n
g
.
L
o
c
k
(
)
 
f
o
r
 
n
 
i
n
 
r
a
n
g
e
(
N
S
T
I
C
K
S
)
]




#
 
C
r
e
a
t
e
 
a
l
l
 
o
f
 
t
h
e
 
p
h
i
l
o
s
o
p
h
e
r
s


f
o
r
 
n
 
i
n
 
r
a
n
g
e
(
N
S
T
I
C
K
S
)
:


 
 
 
 
t
 
=
 
t
h
r
e
a
d
i
n
g
.
T
h
r
e
a
d
(
t
a
r
g
e
t
=
p
h
i
l
o
s
o
p
h
e
r
,


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
a
r
g
s
=
(
c
h
o
p
s
t
i
c
k
s
[
n
]
,
c
h
o
p
s
t
i
c
k
s
[
(
n
+
1
)
 
%
 
N
S
T
I
C
K
S
]
)
)


 
 
 
 
t
.
s
t
a
r
t
(
)



最后，要特别注意到，为了避免死锁，所有的加锁操作必须使用 acquire() 函数。如果代码中的某部分绕过acquire
函数直接申请锁，那么整个死锁避免机制就不起作用了。

## 12.6 保存线程的状态信息


### 问题


你需要保存正在运行线程的状态，这个状态对于其他的线程是不可见的。

### 解决方案


有时在多线程编程中，你需要只保存当前运行线程的状态。
要这么做，可使用 thread.local() 创建一个本地线程存储对象。
对这个对象的属性的保存和读取操作都只会对执行线程可见，而其他线程并不可见。

作为使用本地存储的一个有趣的实际例子，
考虑在8.3小节定义过的 LazyConnection 上下文管理器类。
下面我们对它进行一些小的修改使得它可以适用于多线程：

In [ ]:
f
r
o
m
 
s
o
c
k
e
t
 
i
m
p
o
r
t
 
s
o
c
k
e
t
,
 
A
F
_
I
N
E
T
,
 
S
O
C
K
_
S
T
R
E
A
M


i
m
p
o
r
t
 
t
h
r
e
a
d
i
n
g




c
l
a
s
s
 
L
a
z
y
C
o
n
n
e
c
t
i
o
n
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
a
d
d
r
e
s
s
,
 
f
a
m
i
l
y
=
A
F
_
I
N
E
T
,
 
t
y
p
e
=
S
O
C
K
_
S
T
R
E
A
M
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
a
d
d
r
e
s
s
 
=
 
a
d
d
r
e
s
s


 
 
 
 
 
 
 
 
s
e
l
f
.
f
a
m
i
l
y
 
=
 
A
F
_
I
N
E
T


 
 
 
 
 
 
 
 
s
e
l
f
.
t
y
p
e
 
=
 
S
O
C
K
_
S
T
R
E
A
M


 
 
 
 
 
 
 
 
s
e
l
f
.
l
o
c
a
l
 
=
 
t
h
r
e
a
d
i
n
g
.
l
o
c
a
l
(
)




 
 
 
 
d
e
f
 
_
_
e
n
t
e
r
_
_
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
i
f
 
h
a
s
a
t
t
r
(
s
e
l
f
.
l
o
c
a
l
,
 
'
s
o
c
k
'
)
:


 
 
 
 
 
 
 
 
 
 
 
 
r
a
i
s
e
 
R
u
n
t
i
m
e
E
r
r
o
r
(
'
A
l
r
e
a
d
y
 
c
o
n
n
e
c
t
e
d
'
)


 
 
 
 
 
 
 
 
s
e
l
f
.
l
o
c
a
l
.
s
o
c
k
 
=
 
s
o
c
k
e
t
(
s
e
l
f
.
f
a
m
i
l
y
,
 
s
e
l
f
.
t
y
p
e
)


 
 
 
 
 
 
 
 
s
e
l
f
.
l
o
c
a
l
.
s
o
c
k
.
c
o
n
n
e
c
t
(
s
e
l
f
.
a
d
d
r
e
s
s
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
s
e
l
f
.
l
o
c
a
l
.
s
o
c
k




 
 
 
 
d
e
f
 
_
_
e
x
i
t
_
_
(
s
e
l
f
,
 
e
x
c
_
t
y
,
 
e
x
c
_
v
a
l
,
 
t
b
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
l
o
c
a
l
.
s
o
c
k
.
c
l
o
s
e
(
)


 
 
 
 
 
 
 
 
d
e
l
 
s
e
l
f
.
l
o
c
a
l
.
s
o
c
k



代码中，自己观察对于 self.local 属性的使用。
它被初始化为一个 threading.local() 实例。
其他方法操作被存储为 self.local.sock 的套接字对象。
有了这些就可以在多线程中安全的使用 LazyConnection 实例了。例如：

In [ ]:
f
r
o
m
 
f
u
n
c
t
o
o
l
s
 
i
m
p
o
r
t
 
p
a
r
t
i
a
l


d
e
f
 
t
e
s
t
(
c
o
n
n
)
:


 
 
 
 
w
i
t
h
 
c
o
n
n
 
a
s
 
s
:


 
 
 
 
 
 
 
 
s
.
s
e
n
d
(
b
'
G
E
T
 
/
i
n
d
e
x
.
h
t
m
l
 
H
T
T
P
/
1
.
0
\
r
\
n
'
)


 
 
 
 
 
 
 
 
s
.
s
e
n
d
(
b
'
H
o
s
t
:
 
w
w
w
.
p
y
t
h
o
n
.
o
r
g
\
r
\
n
'
)




 
 
 
 
 
 
 
 
s
.
s
e
n
d
(
b
'
\
r
\
n
'
)


 
 
 
 
 
 
 
 
r
e
s
p
 
=
 
b
'
'
.
j
o
i
n
(
i
t
e
r
(
p
a
r
t
i
a
l
(
s
.
r
e
c
v
,
 
8
1
9
2
)
,
 
b
'
'
)
)




 
 
 
 
p
r
i
n
t
(
'
G
o
t
 
{
}
 
b
y
t
e
s
'
.
f
o
r
m
a
t
(
l
e
n
(
r
e
s
p
)
)
)




i
f
 
_
_
n
a
m
e
_
_
 
=
=
 
'
_
_
m
a
i
n
_
_
'
:


 
 
 
 
c
o
n
n
 
=
 
L
a
z
y
C
o
n
n
e
c
t
i
o
n
(
(
'
w
w
w
.
p
y
t
h
o
n
.
o
r
g
'
,
 
8
0
)
)




 
 
 
 
t
1
 
=
 
t
h
r
e
a
d
i
n
g
.
T
h
r
e
a
d
(
t
a
r
g
e
t
=
t
e
s
t
,
 
a
r
g
s
=
(
c
o
n
n
,
)
)


 
 
 
 
t
2
 
=
 
t
h
r
e
a
d
i
n
g
.
T
h
r
e
a
d
(
t
a
r
g
e
t
=
t
e
s
t
,
 
a
r
g
s
=
(
c
o
n
n
,
)
)


 
 
 
 
t
1
.
s
t
a
r
t
(
)


 
 
 
 
t
2
.
s
t
a
r
t
(
)


 
 
 
 
t
1
.
j
o
i
n
(
)


 
 
 
 
t
2
.
j
o
i
n
(
)



它之所以行得通的原因是每个线程会创建一个自己专属的套接字连接（存储为self.local.sock）。
因此，当不同的线程执行套接字操作时，由于操作的是不同的套接字，因此它们不会相互影响。

### 讨论


在大部分程序中创建和操作线程特定状态并不会有什么问题。
不过，当出了问题的时候，通常是因为某个对象被多个线程使用到，用来操作一些专用的系统资源，
比如一个套接字或文件。你不能让所有线程共享一个单独对象，
因为多个线程同时读和写的时候会产生混乱。
本地线程存储通过让这些资源只能在被使用的线程中可见来解决这个问题。

本节中，使用 thread.local() 可以让 LazyConnection 类支持一个线程一个连接，
而不是对于所有的进程都只有一个连接。

其原理是，每个 threading.local() 实例为每个线程维护着一个单独的实例字典。
所有普通实例操作比如获取、修改和删除值仅仅操作这个字典。
每个线程使用一个独立的字典就可以保证数据的隔离了。

## 12.7 创建一个线程池


### 问题


你创建一个工作者线程池，用来响应客户端请求或执行其他的工作。

### 解决方案


concurrent.futures 函数库有一个 ThreadPoolExecutor 类可以被用来完成这个任务。
下面是一个简单的TCP服务器，使用了一个线程池来响应客户端：

In [ ]:
f
r
o
m
 
s
o
c
k
e
t
 
i
m
p
o
r
t
 
A
F
_
I
N
E
T
,
 
S
O
C
K
_
S
T
R
E
A
M
,
 
s
o
c
k
e
t


f
r
o
m
 
c
o
n
c
u
r
r
e
n
t
.
f
u
t
u
r
e
s
 
i
m
p
o
r
t
 
T
h
r
e
a
d
P
o
o
l
E
x
e
c
u
t
o
r




d
e
f
 
e
c
h
o
_
c
l
i
e
n
t
(
s
o
c
k
,
 
c
l
i
e
n
t
_
a
d
d
r
)
:


 
 
 
 
'
'
'


 
 
 
 
H
a
n
d
l
e
 
a
 
c
l
i
e
n
t
 
c
o
n
n
e
c
t
i
o
n


 
 
 
 
'
'
'


 
 
 
 
p
r
i
n
t
(
'
G
o
t
 
c
o
n
n
e
c
t
i
o
n
 
f
r
o
m
'
,
 
c
l
i
e
n
t
_
a
d
d
r
)


 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
m
s
g
 
=
 
s
o
c
k
.
r
e
c
v
(
6
5
5
3
6
)


 
 
 
 
 
 
 
 
i
f
 
n
o
t
 
m
s
g
:


 
 
 
 
 
 
 
 
 
 
 
 
b
r
e
a
k


 
 
 
 
 
 
 
 
s
o
c
k
.
s
e
n
d
a
l
l
(
m
s
g
)


 
 
 
 
p
r
i
n
t
(
'
C
l
i
e
n
t
 
c
l
o
s
e
d
 
c
o
n
n
e
c
t
i
o
n
'
)


 
 
 
 
s
o
c
k
.
c
l
o
s
e
(
)




d
e
f
 
e
c
h
o
_
s
e
r
v
e
r
(
a
d
d
r
)
:


 
 
 
 
p
o
o
l
 
=
 
T
h
r
e
a
d
P
o
o
l
E
x
e
c
u
t
o
r
(
1
2
8
)


 
 
 
 
s
o
c
k
 
=
 
s
o
c
k
e
t
(
A
F
_
I
N
E
T
,
 
S
O
C
K
_
S
T
R
E
A
M
)


 
 
 
 
s
o
c
k
.
b
i
n
d
(
a
d
d
r
)


 
 
 
 
s
o
c
k
.
l
i
s
t
e
n
(
5
)


 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
c
l
i
e
n
t
_
s
o
c
k
,
 
c
l
i
e
n
t
_
a
d
d
r
 
=
 
s
o
c
k
.
a
c
c
e
p
t
(
)


 
 
 
 
 
 
 
 
p
o
o
l
.
s
u
b
m
i
t
(
e
c
h
o
_
c
l
i
e
n
t
,
 
c
l
i
e
n
t
_
s
o
c
k
,
 
c
l
i
e
n
t
_
a
d
d
r
)




e
c
h
o
_
s
e
r
v
e
r
(
(
'
'
,
1
5
0
0
0
)
)



如果你想手动创建你自己的线程池，
通常可以使用一个Queue来轻松实现。下面是一个稍微不同但是手动实现的例子：

In [ ]:
f
r
o
m
 
s
o
c
k
e
t
 
i
m
p
o
r
t
 
s
o
c
k
e
t
,
 
A
F
_
I
N
E
T
,
 
S
O
C
K
_
S
T
R
E
A
M


f
r
o
m
 
t
h
r
e
a
d
i
n
g
 
i
m
p
o
r
t
 
T
h
r
e
a
d


f
r
o
m
 
q
u
e
u
e
 
i
m
p
o
r
t
 
Q
u
e
u
e




d
e
f
 
e
c
h
o
_
c
l
i
e
n
t
(
q
)
:


 
 
 
 
'
'
'


 
 
 
 
H
a
n
d
l
e
 
a
 
c
l
i
e
n
t
 
c
o
n
n
e
c
t
i
o
n


 
 
 
 
'
'
'


 
 
 
 
s
o
c
k
,
 
c
l
i
e
n
t
_
a
d
d
r
 
=
 
q
.
g
e
t
(
)


 
 
 
 
p
r
i
n
t
(
'
G
o
t
 
c
o
n
n
e
c
t
i
o
n
 
f
r
o
m
'
,
 
c
l
i
e
n
t
_
a
d
d
r
)


 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
m
s
g
 
=
 
s
o
c
k
.
r
e
c
v
(
6
5
5
3
6
)


 
 
 
 
 
 
 
 
i
f
 
n
o
t
 
m
s
g
:


 
 
 
 
 
 
 
 
 
 
 
 
b
r
e
a
k


 
 
 
 
 
 
 
 
s
o
c
k
.
s
e
n
d
a
l
l
(
m
s
g
)


 
 
 
 
p
r
i
n
t
(
'
C
l
i
e
n
t
 
c
l
o
s
e
d
 
c
o
n
n
e
c
t
i
o
n
'
)




 
 
 
 
s
o
c
k
.
c
l
o
s
e
(
)




d
e
f
 
e
c
h
o
_
s
e
r
v
e
r
(
a
d
d
r
,
 
n
w
o
r
k
e
r
s
)
:


 
 
 
 
#
 
L
a
u
n
c
h
 
t
h
e
 
c
l
i
e
n
t
 
w
o
r
k
e
r
s


 
 
 
 
q
 
=
 
Q
u
e
u
e
(
)


 
 
 
 
f
o
r
 
n
 
i
n
 
r
a
n
g
e
(
n
w
o
r
k
e
r
s
)
:


 
 
 
 
 
 
 
 
t
 
=
 
T
h
r
e
a
d
(
t
a
r
g
e
t
=
e
c
h
o
_
c
l
i
e
n
t
,
 
a
r
g
s
=
(
q
,
)
)


 
 
 
 
 
 
 
 
t
.
d
a
e
m
o
n
 
=
 
T
r
u
e


 
 
 
 
 
 
 
 
t
.
s
t
a
r
t
(
)




 
 
 
 
#
 
R
u
n
 
t
h
e
 
s
e
r
v
e
r


 
 
 
 
s
o
c
k
 
=
 
s
o
c
k
e
t
(
A
F
_
I
N
E
T
,
 
S
O
C
K
_
S
T
R
E
A
M
)


 
 
 
 
s
o
c
k
.
b
i
n
d
(
a
d
d
r
)


 
 
 
 
s
o
c
k
.
l
i
s
t
e
n
(
5
)


 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
c
l
i
e
n
t
_
s
o
c
k
,
 
c
l
i
e
n
t
_
a
d
d
r
 
=
 
s
o
c
k
.
a
c
c
e
p
t
(
)


 
 
 
 
 
 
 
 
q
.
p
u
t
(
(
c
l
i
e
n
t
_
s
o
c
k
,
 
c
l
i
e
n
t
_
a
d
d
r
)
)




e
c
h
o
_
s
e
r
v
e
r
(
(
'
'
,
1
5
0
0
0
)
,
 
1
2
8
)



使用 ThreadPoolExecutor 相对于手动实现的一个好处在于它使得
任务提交者更方便的从被调用函数中获取返回值。例如，你可能会像下面这样写：

In [ ]:
f
r
o
m
 
c
o
n
c
u
r
r
e
n
t
.
f
u
t
u
r
e
s
 
i
m
p
o
r
t
 
T
h
r
e
a
d
P
o
o
l
E
x
e
c
u
t
o
r


i
m
p
o
r
t
 
u
r
l
l
i
b
.
r
e
q
u
e
s
t




d
e
f
 
f
e
t
c
h
_
u
r
l
(
u
r
l
)
:


 
 
 
 
u
 
=
 
u
r
l
l
i
b
.
r
e
q
u
e
s
t
.
u
r
l
o
p
e
n
(
u
r
l
)


 
 
 
 
d
a
t
a
 
=
 
u
.
r
e
a
d
(
)


 
 
 
 
r
e
t
u
r
n
 
d
a
t
a




p
o
o
l
 
=
 
T
h
r
e
a
d
P
o
o
l
E
x
e
c
u
t
o
r
(
1
0
)


#
 
S
u
b
m
i
t
 
w
o
r
k
 
t
o
 
t
h
e
 
p
o
o
l


a
 
=
 
p
o
o
l
.
s
u
b
m
i
t
(
f
e
t
c
h
_
u
r
l
,
 
'
h
t
t
p
:
/
/
w
w
w
.
p
y
t
h
o
n
.
o
r
g
'
)


b
 
=
 
p
o
o
l
.
s
u
b
m
i
t
(
f
e
t
c
h
_
u
r
l
,
 
'
h
t
t
p
:
/
/
w
w
w
.
p
y
p
y
.
o
r
g
'
)




#
 
G
e
t
 
t
h
e
 
r
e
s
u
l
t
s
 
b
a
c
k


x
 
=
 
a
.
r
e
s
u
l
t
(
)


y
 
=
 
b
.
r
e
s
u
l
t
(
)



例子中返回的handle对象会帮你处理所有的阻塞与协作，然后从工作线程中返回数据给你。
特别的，a.result() 操作会阻塞进程直到对应的函数执行完成并返回一个结果。

### 讨论


通常来讲，你应该避免编写线程数量可以无限制增长的程序。例如，看看下面这个服务器：

In [ ]:
f
r
o
m
 
t
h
r
e
a
d
i
n
g
 
i
m
p
o
r
t
 
T
h
r
e
a
d


f
r
o
m
 
s
o
c
k
e
t
 
i
m
p
o
r
t
 
s
o
c
k
e
t
,
 
A
F
_
I
N
E
T
,
 
S
O
C
K
_
S
T
R
E
A
M




d
e
f
 
e
c
h
o
_
c
l
i
e
n
t
(
s
o
c
k
,
 
c
l
i
e
n
t
_
a
d
d
r
)
:


 
 
 
 
'
'
'


 
 
 
 
H
a
n
d
l
e
 
a
 
c
l
i
e
n
t
 
c
o
n
n
e
c
t
i
o
n


 
 
 
 
'
'
'


 
 
 
 
p
r
i
n
t
(
'
G
o
t
 
c
o
n
n
e
c
t
i
o
n
 
f
r
o
m
'
,
 
c
l
i
e
n
t
_
a
d
d
r
)


 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
m
s
g
 
=
 
s
o
c
k
.
r
e
c
v
(
6
5
5
3
6
)


 
 
 
 
 
 
 
 
i
f
 
n
o
t
 
m
s
g
:


 
 
 
 
 
 
 
 
 
 
 
 
b
r
e
a
k


 
 
 
 
 
 
 
 
s
o
c
k
.
s
e
n
d
a
l
l
(
m
s
g
)


 
 
 
 
p
r
i
n
t
(
'
C
l
i
e
n
t
 
c
l
o
s
e
d
 
c
o
n
n
e
c
t
i
o
n
'
)


 
 
 
 
s
o
c
k
.
c
l
o
s
e
(
)




d
e
f
 
e
c
h
o
_
s
e
r
v
e
r
(
a
d
d
r
,
 
n
w
o
r
k
e
r
s
)
:


 
 
 
 
#
 
R
u
n
 
t
h
e
 
s
e
r
v
e
r


 
 
 
 
s
o
c
k
 
=
 
s
o
c
k
e
t
(
A
F
_
I
N
E
T
,
 
S
O
C
K
_
S
T
R
E
A
M
)


 
 
 
 
s
o
c
k
.
b
i
n
d
(
a
d
d
r
)


 
 
 
 
s
o
c
k
.
l
i
s
t
e
n
(
5
)


 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
c
l
i
e
n
t
_
s
o
c
k
,
 
c
l
i
e
n
t
_
a
d
d
r
 
=
 
s
o
c
k
.
a
c
c
e
p
t
(
)


 
 
 
 
 
 
 
 
t
 
=
 
T
h
r
e
a
d
(
t
a
r
g
e
t
=
e
c
h
o
_
c
l
i
e
n
t
,
 
a
r
g
s
=
(
c
l
i
e
n
t
_
s
o
c
k
,
 
c
l
i
e
n
t
_
a
d
d
r
)
)


 
 
 
 
 
 
 
 
t
.
d
a
e
m
o
n
 
=
 
T
r
u
e


 
 
 
 
 
 
 
 
t
.
s
t
a
r
t
(
)




e
c
h
o
_
s
e
r
v
e
r
(
(
'
'
,
1
5
0
0
0
)
)



尽管这个也可以工作，
但是它不能抵御有人试图通过创建大量线程让你服务器资源枯竭而崩溃的攻击行为。
通过使用预先初始化的线程池，你可以设置同时运行线程的上限数量。

你可能会关心创建大量线程会有什么后果。
现代操作系统可以很轻松的创建几千个线程的线程池。
甚至，同时几千个线程等待工作并不会对其他代码产生性能影响。
当然了，如果所有线程同时被唤醒并立即在CPU上执行，那就不同了——特别是有了全局解释器锁GIL。
通常，你应该只在I/O处理相关代码中使用线程池。

创建大的线程池的一个可能需要关注的问题是内存的使用。
例如，如果你在OS X系统上面创建2000个线程，系统显示Python进程使用了超过9GB的虚拟内存。
不过，这个计算通常是有误差的。当创建一个线程时，操作系统会预留一个虚拟内存区域来
放置线程的执行栈（通常是8MB大小）。但是这个内存只有一小片段被实际映射到真实内存中。
因此，Python进程使用到的真实内存其实很小
（比如，对于2000个线程来讲，只使用到了70MB的真实内存，而不是9GB）。
如果你担心虚拟内存大小，可以使用 threading.stack_size() 函数来降低它。例如：

In [ ]:
i
m
p
o
r
t
 
t
h
r
e
a
d
i
n
g


t
h
r
e
a
d
i
n
g
.
s
t
a
c
k
_
s
i
z
e
(
6
5
5
3
6
)



如果你加上这条语句并再次运行前面的创建2000个线程试验，
你会发现Python进程只使用到了大概210MB的虚拟内存，而真实内存使用量没有变。
注意线程栈大小必须至少为32768字节，通常是系统内存页大小（4096、8192等）的整数倍。

## 12.8 简单的并行编程


### 问题


你有个程序要执行CPU密集型工作，你想让他利用多核CPU的优势来运行的快一点。

### 解决方案


concurrent.futures 库提供了一个 ProcessPoolExecutor 类，
可被用来在一个单独的Python解释器中执行计算密集型函数。
不过，要使用它，你首先要有一些计算密集型的任务。
我们通过一个简单而实际的例子来演示它。假定你有个Apache web服务器日志目录的gzip压缩包：

In [ ]:
l
o
g
s
/


 
 
 
2
0
1
2
0
7
0
1
.
l
o
g
.
g
z


 
 
 
2
0
1
2
0
7
0
2
.
l
o
g
.
g
z


 
 
 
2
0
1
2
0
7
0
3
.
l
o
g
.
g
z


 
 
 
2
0
1
2
0
7
0
4
.
l
o
g
.
g
z


 
 
 
2
0
1
2
0
7
0
5
.
l
o
g
.
g
z


 
 
 
2
0
1
2
0
7
0
6
.
l
o
g
.
g
z


 
 
 
.
.
.



进一步假设每个日志文件内容类似下面这样：

In [ ]:
1
2
4
.
1
1
5
.
6
.
1
2
 
-
 
-
 
[
1
0
/
J
u
l
/
2
0
1
2
:
0
0
:
1
8
:
5
0
 
-
0
5
0
0
]
 
"
G
E
T
 
/
r
o
b
o
t
s
.
t
x
t
 
.
.
.
"
 
2
0
0
 
7
1


2
1
0
.
2
1
2
.
2
0
9
.
6
7
 
-
 
-
 
[
1
0
/
J
u
l
/
2
0
1
2
:
0
0
:
1
8
:
5
1
 
-
0
5
0
0
]
 
"
G
E
T
 
/
p
l
y
/
 
.
.
.
"
 
2
0
0
 
1
1
8
7
5


2
1
0
.
2
1
2
.
2
0
9
.
6
7
 
-
 
-
 
[
1
0
/
J
u
l
/
2
0
1
2
:
0
0
:
1
8
:
5
1
 
-
0
5
0
0
]
 
"
G
E
T
 
/
f
a
v
i
c
o
n
.
i
c
o
 
.
.
.
"
 
4
0
4
 
3
6
9


6
1
.
1
3
5
.
2
1
6
.
1
0
5
 
-
 
-
 
[
1
0
/
J
u
l
/
2
0
1
2
:
0
0
:
2
0
:
0
4
 
-
0
5
0
0
]
 
"
G
E
T
 
/
b
l
o
g
/
a
t
o
m
.
x
m
l
 
.
.
.
"
 
3
0
4
 
-


.
.
.



下面是一个脚本，在这些日志文件中查找出所有访问过robots.txt文件的主机：

In [ ]:
#
 
f
i
n
d
r
o
b
o
t
s
.
p
y




i
m
p
o
r
t
 
g
z
i
p


i
m
p
o
r
t
 
i
o


i
m
p
o
r
t
 
g
l
o
b




d
e
f
 
f
i
n
d
_
r
o
b
o
t
s
(
f
i
l
e
n
a
m
e
)
:


 
 
 
 
'
'
'


 
 
 
 
F
i
n
d
 
a
l
l
 
o
f
 
t
h
e
 
h
o
s
t
s
 
t
h
a
t
 
a
c
c
e
s
s
 
r
o
b
o
t
s
.
t
x
t
 
i
n
 
a
 
s
i
n
g
l
e
 
l
o
g
 
f
i
l
e


 
 
 
 
'
'
'


 
 
 
 
r
o
b
o
t
s
 
=
 
s
e
t
(
)


 
 
 
 
w
i
t
h
 
g
z
i
p
.
o
p
e
n
(
f
i
l
e
n
a
m
e
)
 
a
s
 
f
:


 
 
 
 
 
 
 
 
f
o
r
 
l
i
n
e
 
i
n
 
i
o
.
T
e
x
t
I
O
W
r
a
p
p
e
r
(
f
,
e
n
c
o
d
i
n
g
=
'
a
s
c
i
i
'
)
:


 
 
 
 
 
 
 
 
 
 
 
 
f
i
e
l
d
s
 
=
 
l
i
n
e
.
s
p
l
i
t
(
)


 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
f
i
e
l
d
s
[
6
]
 
=
=
 
'
/
r
o
b
o
t
s
.
t
x
t
'
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
r
o
b
o
t
s
.
a
d
d
(
f
i
e
l
d
s
[
0
]
)


 
 
 
 
r
e
t
u
r
n
 
r
o
b
o
t
s




d
e
f
 
f
i
n
d
_
a
l
l
_
r
o
b
o
t
s
(
l
o
g
d
i
r
)
:


 
 
 
 
'
'
'


 
 
 
 
F
i
n
d
 
a
l
l
 
h
o
s
t
s
 
a
c
r
o
s
s
 
a
n
d
 
e
n
t
i
r
e
 
s
e
q
u
e
n
c
e
 
o
f
 
f
i
l
e
s


 
 
 
 
'
'
'


 
 
 
 
f
i
l
e
s
 
=
 
g
l
o
b
.
g
l
o
b
(
l
o
g
d
i
r
+
'
/
*
.
l
o
g
.
g
z
'
)


 
 
 
 
a
l
l
_
r
o
b
o
t
s
 
=
 
s
e
t
(
)


 
 
 
 
f
o
r
 
r
o
b
o
t
s
 
i
n
 
m
a
p
(
f
i
n
d
_
r
o
b
o
t
s
,
 
f
i
l
e
s
)
:


 
 
 
 
 
 
 
 
a
l
l
_
r
o
b
o
t
s
.
u
p
d
a
t
e
(
r
o
b
o
t
s
)


 
 
 
 
r
e
t
u
r
n
 
a
l
l
_
r
o
b
o
t
s




i
f
 
_
_
n
a
m
e
_
_
 
=
=
 
'
_
_
m
a
i
n
_
_
'
:


 
 
 
 
r
o
b
o
t
s
 
=
 
f
i
n
d
_
a
l
l
_
r
o
b
o
t
s
(
'
l
o
g
s
'
)


 
 
 
 
f
o
r
 
i
p
a
d
d
r
 
i
n
 
r
o
b
o
t
s
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
i
p
a
d
d
r
)



前面的程序使用了通常的map-reduce风格来编写。
函数 find_robots() 在一个文件名集合上做map操作，并将结果汇总为一个单独的结果，
也就是 find_all_robots() 函数中的 all_robots 集合。
现在，假设你想要修改这个程序让它使用多核CPU。
很简单——只需要将map()操作替换为一个 concurrent.futures 库中生成的类似操作即可。
下面是一个简单修改版本：

In [ ]:
#
 
f
i
n
d
r
o
b
o
t
s
.
p
y




i
m
p
o
r
t
 
g
z
i
p


i
m
p
o
r
t
 
i
o


i
m
p
o
r
t
 
g
l
o
b


f
r
o
m
 
c
o
n
c
u
r
r
e
n
t
 
i
m
p
o
r
t
 
f
u
t
u
r
e
s




d
e
f
 
f
i
n
d
_
r
o
b
o
t
s
(
f
i
l
e
n
a
m
e
)
:


 
 
 
 
'
'
'


 
 
 
 
F
i
n
d
 
a
l
l
 
o
f
 
t
h
e
 
h
o
s
t
s
 
t
h
a
t
 
a
c
c
e
s
s
 
r
o
b
o
t
s
.
t
x
t
 
i
n
 
a
 
s
i
n
g
l
e
 
l
o
g
 
f
i
l
e




 
 
 
 
'
'
'


 
 
 
 
r
o
b
o
t
s
 
=
 
s
e
t
(
)


 
 
 
 
w
i
t
h
 
g
z
i
p
.
o
p
e
n
(
f
i
l
e
n
a
m
e
)
 
a
s
 
f
:


 
 
 
 
 
 
 
 
f
o
r
 
l
i
n
e
 
i
n
 
i
o
.
T
e
x
t
I
O
W
r
a
p
p
e
r
(
f
,
e
n
c
o
d
i
n
g
=
'
a
s
c
i
i
'
)
:


 
 
 
 
 
 
 
 
 
 
 
 
f
i
e
l
d
s
 
=
 
l
i
n
e
.
s
p
l
i
t
(
)


 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
f
i
e
l
d
s
[
6
]
 
=
=
 
'
/
r
o
b
o
t
s
.
t
x
t
'
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
r
o
b
o
t
s
.
a
d
d
(
f
i
e
l
d
s
[
0
]
)


 
 
 
 
r
e
t
u
r
n
 
r
o
b
o
t
s




d
e
f
 
f
i
n
d
_
a
l
l
_
r
o
b
o
t
s
(
l
o
g
d
i
r
)
:


 
 
 
 
'
'
'


 
 
 
 
F
i
n
d
 
a
l
l
 
h
o
s
t
s
 
a
c
r
o
s
s
 
a
n
d
 
e
n
t
i
r
e
 
s
e
q
u
e
n
c
e
 
o
f
 
f
i
l
e
s


 
 
 
 
'
'
'


 
 
 
 
f
i
l
e
s
 
=
 
g
l
o
b
.
g
l
o
b
(
l
o
g
d
i
r
+
'
/
*
.
l
o
g
.
g
z
'
)


 
 
 
 
a
l
l
_
r
o
b
o
t
s
 
=
 
s
e
t
(
)


 
 
 
 
w
i
t
h
 
f
u
t
u
r
e
s
.
P
r
o
c
e
s
s
P
o
o
l
E
x
e
c
u
t
o
r
(
)
 
a
s
 
p
o
o
l
:


 
 
 
 
 
 
 
 
f
o
r
 
r
o
b
o
t
s
 
i
n
 
p
o
o
l
.
m
a
p
(
f
i
n
d
_
r
o
b
o
t
s
,
 
f
i
l
e
s
)
:


 
 
 
 
 
 
 
 
 
 
 
 
a
l
l
_
r
o
b
o
t
s
.
u
p
d
a
t
e
(
r
o
b
o
t
s
)


 
 
 
 
r
e
t
u
r
n
 
a
l
l
_
r
o
b
o
t
s




i
f
 
_
_
n
a
m
e
_
_
 
=
=
 
'
_
_
m
a
i
n
_
_
'
:


 
 
 
 
r
o
b
o
t
s
 
=
 
f
i
n
d
_
a
l
l
_
r
o
b
o
t
s
(
'
l
o
g
s
'
)


 
 
 
 
f
o
r
 
i
p
a
d
d
r
 
i
n
 
r
o
b
o
t
s
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
i
p
a
d
d
r
)



通过这个修改后，运行这个脚本产生同样的结果，但是在四核机器上面比之前快了3.5倍。
实际的性能优化效果根据你的机器CPU数量的不同而不同。

### 讨论


ProcessPoolExecutor 的典型用法如下：

In [ ]:
f
r
o
m
 
c
o
n
c
u
r
r
e
n
t
.
f
u
t
u
r
e
s
 
i
m
p
o
r
t
 
P
r
o
c
e
s
s
P
o
o
l
E
x
e
c
u
t
o
r




w
i
t
h
 
P
r
o
c
e
s
s
P
o
o
l
E
x
e
c
u
t
o
r
(
)
 
a
s
 
p
o
o
l
:


 
 
 
 
.
.
.


 
 
 
 
d
o
 
w
o
r
k
 
i
n
 
p
a
r
a
l
l
e
l
 
u
s
i
n
g
 
p
o
o
l


 
 
 
 
.
.
.



其原理是，一个 ProcessPoolExecutor 创建N个独立的Python解释器，
N是系统上面可用CPU的个数。你可以通过提供可选参数给 ProcessPoolExecutor(N) 来修改
处理器数量。这个处理池会一直运行到with块中最后一个语句执行完成，
然后处理池被关闭。不过，程序会一直等待直到所有提交的工作被处理完成。

被提交到池中的工作必须被定义为一个函数。有两种方法去提交。
如果你想让一个列表推导或一个 map() 操作并行执行的话，可使用 pool.map() :

In [ ]:
#
 
A
 
f
u
n
c
t
i
o
n
 
t
h
a
t
 
p
e
r
f
o
r
m
s
 
a
 
l
o
t
 
o
f
 
w
o
r
k


d
e
f
 
w
o
r
k
(
x
)
:


 
 
 
 
.
.
.


 
 
 
 
r
e
t
u
r
n
 
r
e
s
u
l
t




#
 
N
o
n
p
a
r
a
l
l
e
l
 
c
o
d
e


r
e
s
u
l
t
s
 
=
 
m
a
p
(
w
o
r
k
,
 
d
a
t
a
)




#
 
P
a
r
a
l
l
e
l
 
i
m
p
l
e
m
e
n
t
a
t
i
o
n


w
i
t
h
 
P
r
o
c
e
s
s
P
o
o
l
E
x
e
c
u
t
o
r
(
)
 
a
s
 
p
o
o
l
:


 
 
 
 
r
e
s
u
l
t
s
 
=
 
p
o
o
l
.
m
a
p
(
w
o
r
k
,
 
d
a
t
a
)



另外，你可以使用 pool.submit() 来手动的提交单个任务：

In [ ]:
#
 
S
o
m
e
 
f
u
n
c
t
i
o
n


d
e
f
 
w
o
r
k
(
x
)
:


 
 
 
 
.
.
.


 
 
 
 
r
e
t
u
r
n
 
r
e
s
u
l
t




w
i
t
h
 
P
r
o
c
e
s
s
P
o
o
l
E
x
e
c
u
t
o
r
(
)
 
a
s
 
p
o
o
l
:


 
 
 
 
.
.
.


 
 
 
 
#
 
E
x
a
m
p
l
e
 
o
f
 
s
u
b
m
i
t
t
i
n
g
 
w
o
r
k
 
t
o
 
t
h
e
 
p
o
o
l


 
 
 
 
f
u
t
u
r
e
_
r
e
s
u
l
t
 
=
 
p
o
o
l
.
s
u
b
m
i
t
(
w
o
r
k
,
 
a
r
g
)




 
 
 
 
#
 
O
b
t
a
i
n
i
n
g
 
t
h
e
 
r
e
s
u
l
t
 
(
b
l
o
c
k
s
 
u
n
t
i
l
 
d
o
n
e
)


 
 
 
 
r
 
=
 
f
u
t
u
r
e
_
r
e
s
u
l
t
.
r
e
s
u
l
t
(
)


 
 
 
 
.
.
.



如果你手动提交一个任务，结果是一个 Future 实例。
要获取最终结果，你需要调用它的 result() 方法。
它会阻塞进程直到结果被返回来。

如果不想阻塞，你还可以使用一个回调函数，例如：

In [ ]:
d
e
f
 
w
h
e
n
_
d
o
n
e
(
r
)
:


 
 
 
 
p
r
i
n
t
(
'
G
o
t
:
'
,
 
r
.
r
e
s
u
l
t
(
)
)




w
i
t
h
 
P
r
o
c
e
s
s
P
o
o
l
E
x
e
c
u
t
o
r
(
)
 
a
s
 
p
o
o
l
:


 
 
 
 
 
f
u
t
u
r
e
_
r
e
s
u
l
t
 
=
 
p
o
o
l
.
s
u
b
m
i
t
(
w
o
r
k
,
 
a
r
g
)


 
 
 
 
 
f
u
t
u
r
e
_
r
e
s
u
l
t
.
a
d
d
_
d
o
n
e
_
c
a
l
l
b
a
c
k
(
w
h
e
n
_
d
o
n
e
)



回调函数接受一个 Future 实例，被用来获取最终的结果（比如通过调用它的result()方法）。
尽管处理池很容易使用，在设计大程序的时候还是有很多需要注意的地方，如下几点：

一旦启动你不能控制子进程的任何行为，因此最好保持简单和纯洁——函数不要去修改环境。

它会克隆Python解释器，包括fork时的所有程序状态。
而在Windows上，克隆解释器时不会克隆状态。
实际的fork操作会在第一次调用 pool.map() 或 pool.submit() 后发生。

你应该在创建任何线程之前先创建并激活进程池（比如在程序启动的main线程中创建进程池）。

## 12.9 Python的全局锁问题


### 问题


你已经听说过全局解释器锁GIL，担心它会影响到多线程程序的执行性能。

### 解决方案


尽管Python完全支持多线程编程，
但是解释器的C语言实现部分在完全并行执行时并不是线程安全的。
实际上，解释器被一个全局解释器锁保护着，它确保任何时候都只有一个Python线程执行。
GIL最大的问题就是Python的多线程程序并不能利用多核CPU的优势
（比如一个使用了多个线程的计算密集型程序只会在一个单CPU上面运行）。

在讨论普通的GIL之前，有一点要强调的是GIL只会影响到那些严重依赖CPU的程序（比如计算型的）。
如果你的程序大部分只会涉及到I/O，比如网络交互，那么使用多线程就很合适，
因为它们大部分时间都在等待。实际上，你完全可以放心的创建几千个Python线程，
现代操作系统运行这么多线程没有任何压力，没啥可担心的。

而对于依赖CPU的程序，你需要弄清楚执行的计算的特点。
例如，优化底层算法要比使用多线程运行快得多。
类似的，由于Python是解释执行的，如果你将那些性能瓶颈代码移到一个C语言扩展模块中，
速度也会提升的很快。如果你要操作数组，那么使用NumPy这样的扩展会非常的高效。
最后，你还可以考虑下其他可选实现方案，比如PyPy，它通过一个JIT编译器来优化执行效率
（不过在写这本书的时候它还不能支持Python 3）。

还有一点要注意的是，线程不是专门用来优化性能的。
一个CPU依赖型程序可能会使用线程来管理一个图形用户界面、一个网络连接或其他服务。
这时候，GIL会产生一些问题，因为如果一个线程长期持有GIL的话会导致其他非CPU型线程一直等待。
事实上，一个写的不好的C语言扩展会导致这个问题更加严重，
尽管代码的计算部分会比之前运行的更快些。

说了这么多，现在想说的是我们有两种策略来解决GIL的缺点。
首先，如果你完全工作于Python环境中，你可以使用 multiprocessing 模块来创建一个进程池，
并像协同处理器一样的使用它。例如，假如你有如下的线程代码：

In [ ]:
#
 
P
e
r
f
o
r
m
s
 
a
 
l
a
r
g
e
 
c
a
l
c
u
l
a
t
i
o
n
 
(
C
P
U
 
b
o
u
n
d
)


d
e
f
 
s
o
m
e
_
w
o
r
k
(
a
r
g
s
)
:


 
 
 
 
.
.
.


 
 
 
 
r
e
t
u
r
n
 
r
e
s
u
l
t




#
 
A
 
t
h
r
e
a
d
 
t
h
a
t
 
c
a
l
l
s
 
t
h
e
 
a
b
o
v
e
 
f
u
n
c
t
i
o
n


d
e
f
 
s
o
m
e
_
t
h
r
e
a
d
(
)
:


 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
.
.
.


 
 
 
 
 
 
 
 
r
 
=
 
s
o
m
e
_
w
o
r
k
(
a
r
g
s
)


 
 
 
 
.
.
.



修改代码，使用进程池：

In [ ]:
#
 
P
r
o
c
e
s
s
i
n
g
 
p
o
o
l
 
(
s
e
e
 
b
e
l
o
w
 
f
o
r
 
i
n
i
t
i
a
z
a
t
i
o
n
)


p
o
o
l
 
=
 
N
o
n
e




#
 
P
e
r
f
o
r
m
s
 
a
 
l
a
r
g
e
 
c
a
l
c
u
l
a
t
i
o
n
 
(
C
P
U
 
b
o
u
n
d
)


d
e
f
 
s
o
m
e
_
w
o
r
k
(
a
r
g
s
)
:


 
 
 
 
.
.
.


 
 
 
 
r
e
t
u
r
n
 
r
e
s
u
l
t




#
 
A
 
t
h
r
e
a
d
 
t
h
a
t
 
c
a
l
l
s
 
t
h
e
 
a
b
o
v
e
 
f
u
n
c
t
i
o
n


d
e
f
 
s
o
m
e
_
t
h
r
e
a
d
(
)
:


 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
.
.
.


 
 
 
 
 
 
 
 
r
 
=
 
p
o
o
l
.
a
p
p
l
y
(
s
o
m
e
_
w
o
r
k
,
 
(
a
r
g
s
)
)


 
 
 
 
 
 
 
 
.
.
.




#
 
I
n
i
t
i
a
z
e
 
t
h
e
 
p
o
o
l


i
f
 
_
_
n
a
m
e
_
_
 
=
=
 
'
_
_
m
a
i
n
_
_
'
:


 
 
 
 
i
m
p
o
r
t
 
m
u
l
t
i
p
r
o
c
e
s
s
i
n
g


 
 
 
 
p
o
o
l
 
=
 
m
u
l
t
i
p
r
o
c
e
s
s
i
n
g
.
P
o
o
l
(
)



这个通过使用一个技巧利用进程池解决了GIL的问题。
当一个线程想要执行CPU密集型工作时，会将任务发给进程池。
然后进程池会在另外一个进程中启动一个单独的Python解释器来工作。
当线程等待结果的时候会释放GIL。
并且，由于计算任务在单独解释器中执行，那么就不会受限于GIL了。
在一个多核系统上面，你会发现这个技术可以让你很好的利用多CPU的优势。

另外一个解决GIL的策略是使用C扩展编程技术。
主要思想是将计算密集型任务转移给C，跟Python独立，在工作的时候在C代码中释放GIL。
这可以通过在C代码中插入下面这样的特殊宏来完成：

In [ ]:
#
i
n
c
l
u
d
e
 
"
P
y
t
h
o
n
.
h
"


.
.
.




P
y
O
b
j
e
c
t
 
*
p
y
f
u
n
c
(
P
y
O
b
j
e
c
t
 
*
s
e
l
f
,
 
P
y
O
b
j
e
c
t
 
*
a
r
g
s
)
 
{


 
 
 
.
.
.


 
 
 
P
y
_
B
E
G
I
N
_
A
L
L
O
W
_
T
H
R
E
A
D
S


 
 
 
/
/
 
T
h
r
e
a
d
e
d
 
C
 
c
o
d
e


 
 
 
.
.
.


 
 
 
P
y
_
E
N
D
_
A
L
L
O
W
_
T
H
R
E
A
D
S


 
 
 
.
.
.


}



如果你使用其他工具访问C语言，比如对于Cython的ctypes库，你不需要做任何事。
例如，ctypes在调用C时会自动释放GIL。

### 讨论


许多程序员在面对线程性能问题的时候，马上就会怪罪GIL，什么都是它的问题。
其实这样子太不厚道也太天真了点。
作为一个真实的例子，在多线程的网络编程中神秘的 stalls
可能是因为其他原因比如一个DNS查找延时，而跟GIL毫无关系。
最后你真的需要先去搞懂你的代码是否真的被GIL影响到。
同时还要明白GIL大部分都应该只关注CPU的处理而不是I/O.

如果你准备使用一个处理器池，注意的是这样做涉及到数据序列化和在不同Python解释器通信。
被执行的操作需要放在一个通过def语句定义的Python函数中，不能是lambda、闭包可调用实例等，
并且函数参数和返回值必须要兼容pickle。
同样，要执行的任务量必须足够大以弥补额外的通信开销。

另外一个难点是当混合使用线程和进程池的时候会让你很头疼。
如果你要同时使用两者，最好在程序启动时，创建任何线程之前先创建一个单例的进程池。
然后线程使用同样的进程池来进行它们的计算密集型工作。

C扩展最重要的特征是它们和Python解释器是保持独立的。
也就是说，如果你准备将Python中的任务分配到C中去执行，
你需要确保C代码的操作跟Python保持独立，
这就意味着不要使用Python数据结构以及不要调用Python的C API。
另外一个就是你要确保C扩展所做的工作是足够的，值得你这样做。
也就是说C扩展担负起了大量的计算任务，而不是少数几个计算。

这些解决GIL的方案并不能适用于所有问题。
例如，某些类型的应用程序如果被分解为多个进程处理的话并不能很好的工作，
也不能将它的部分代码改成C语言执行。
对于这些应用程序，你就要自己需求解决方案了
（比如多进程访问共享内存区，多解析器运行于同一个进程等）。
或者，你还可以考虑下其他的解释器实现，比如PyPy。

了解更多关于在C扩展中释放GIL，请参考15.7和15.10小节。

## 12.10 定义一个Actor任务


### 问题


你想定义跟actor模式中类似“actors”角色的任务

### 解决方案


actor模式是一种最古老的也是最简单的并行和分布式计算解决方案。
事实上，它天生的简单性是它如此受欢迎的重要原因之一。
简单来讲，一个actor就是一个并发执行的任务，只是简单的执行发送给它的消息任务。
响应这些消息时，它可能还会给其他actor发送更进一步的消息。
actor之间的通信是单向和异步的。因此，消息发送者不知道消息是什么时候被发送，
也不会接收到一个消息已被处理的回应或通知。

结合使用一个线程和一个队列可以很容易的定义actor，例如：

In [ ]:
f
r
o
m
 
q
u
e
u
e
 
i
m
p
o
r
t
 
Q
u
e
u
e


f
r
o
m
 
t
h
r
e
a
d
i
n
g
 
i
m
p
o
r
t
 
T
h
r
e
a
d
,
 
E
v
e
n
t




#
 
S
e
n
t
i
n
e
l
 
u
s
e
d
 
f
o
r
 
s
h
u
t
d
o
w
n


c
l
a
s
s
 
A
c
t
o
r
E
x
i
t
(
E
x
c
e
p
t
i
o
n
)
:


 
 
 
 
p
a
s
s




c
l
a
s
s
 
A
c
t
o
r
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
m
a
i
l
b
o
x
 
=
 
Q
u
e
u
e
(
)




 
 
 
 
d
e
f
 
s
e
n
d
(
s
e
l
f
,
 
m
s
g
)
:


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
S
e
n
d
 
a
 
m
e
s
s
a
g
e
 
t
o
 
t
h
e
 
a
c
t
o
r


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
s
e
l
f
.
_
m
a
i
l
b
o
x
.
p
u
t
(
m
s
g
)




 
 
 
 
d
e
f
 
r
e
c
v
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
R
e
c
e
i
v
e
 
a
n
 
i
n
c
o
m
i
n
g
 
m
e
s
s
a
g
e


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
m
s
g
 
=
 
s
e
l
f
.
_
m
a
i
l
b
o
x
.
g
e
t
(
)


 
 
 
 
 
 
 
 
i
f
 
m
s
g
 
i
s
 
A
c
t
o
r
E
x
i
t
:


 
 
 
 
 
 
 
 
 
 
 
 
r
a
i
s
e
 
A
c
t
o
r
E
x
i
t
(
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
m
s
g




 
 
 
 
d
e
f
 
c
l
o
s
e
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
C
l
o
s
e
 
t
h
e
 
a
c
t
o
r
,
 
t
h
u
s
 
s
h
u
t
t
i
n
g
 
i
t
 
d
o
w
n


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
s
e
l
f
.
s
e
n
d
(
A
c
t
o
r
E
x
i
t
)




 
 
 
 
d
e
f
 
s
t
a
r
t
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
S
t
a
r
t
 
c
o
n
c
u
r
r
e
n
t
 
e
x
e
c
u
t
i
o
n


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
s
e
l
f
.
_
t
e
r
m
i
n
a
t
e
d
 
=
 
E
v
e
n
t
(
)


 
 
 
 
 
 
 
 
t
 
=
 
T
h
r
e
a
d
(
t
a
r
g
e
t
=
s
e
l
f
.
_
b
o
o
t
s
t
r
a
p
)




 
 
 
 
 
 
 
 
t
.
d
a
e
m
o
n
 
=
 
T
r
u
e


 
 
 
 
 
 
 
 
t
.
s
t
a
r
t
(
)




 
 
 
 
d
e
f
 
_
b
o
o
t
s
t
r
a
p
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
t
r
y
:


 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
r
u
n
(
)


 
 
 
 
 
 
 
 
e
x
c
e
p
t
 
A
c
t
o
r
E
x
i
t
:


 
 
 
 
 
 
 
 
 
 
 
 
p
a
s
s


 
 
 
 
 
 
 
 
f
i
n
a
l
l
y
:


 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
t
e
r
m
i
n
a
t
e
d
.
s
e
t
(
)




 
 
 
 
d
e
f
 
j
o
i
n
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
t
e
r
m
i
n
a
t
e
d
.
w
a
i
t
(
)




 
 
 
 
d
e
f
 
r
u
n
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
R
u
n
 
m
e
t
h
o
d
 
t
o
 
b
e
 
i
m
p
l
e
m
e
n
t
e
d
 
b
y
 
t
h
e
 
u
s
e
r


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
 
 
 
 
m
s
g
 
=
 
s
e
l
f
.
r
e
c
v
(
)




#
 
S
a
m
p
l
e
 
A
c
t
o
r
T
a
s
k


c
l
a
s
s
 
P
r
i
n
t
A
c
t
o
r
(
A
c
t
o
r
)
:


 
 
 
 
d
e
f
 
r
u
n
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
 
 
 
 
m
s
g
 
=
 
s
e
l
f
.
r
e
c
v
(
)


 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
G
o
t
:
'
,
 
m
s
g
)




#
 
S
a
m
p
l
e
 
u
s
e


p
 
=
 
P
r
i
n
t
A
c
t
o
r
(
)


p
.
s
t
a
r
t
(
)


p
.
s
e
n
d
(
'
H
e
l
l
o
'
)


p
.
s
e
n
d
(
'
W
o
r
l
d
'
)


p
.
c
l
o
s
e
(
)


p
.
j
o
i
n
(
)



这个例子中，你使用actor实例的 send() 方法发送消息给它们。
其机制是，这个方法会将消息放入一个队里中，
然后将其转交给处理被接受消息的一个内部线程。
close() 方法通过在队列中放入一个特殊的哨兵值（ActorExit）来关闭这个actor。
用户可以通过继承Actor并定义实现自己处理逻辑run()方法来定义新的actor。
ActorExit 异常的使用就是用户自定义代码可以在需要的时候来捕获终止请求
（异常被get()方法抛出并传播出去）。

如果你放宽对于同步和异步消息发送的要求，
类actor对象还可以通过生成器来简化定义。例如：

In [ ]:
d
e
f
 
p
r
i
n
t
_
a
c
t
o
r
(
)
:


 
 
 
 
w
h
i
l
e
 
T
r
u
e
:




 
 
 
 
 
 
 
 
t
r
y
:


 
 
 
 
 
 
 
 
 
 
 
 
m
s
g
 
=
 
y
i
e
l
d
 
 
 
 
 
 
#
 
G
e
t
 
a
 
m
e
s
s
a
g
e


 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
G
o
t
:
'
,
 
m
s
g
)


 
 
 
 
 
 
 
 
e
x
c
e
p
t
 
G
e
n
e
r
a
t
o
r
E
x
i
t
:


 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
A
c
t
o
r
 
t
e
r
m
i
n
a
t
i
n
g
'
)




#
 
S
a
m
p
l
e
 
u
s
e


p
 
=
 
p
r
i
n
t
_
a
c
t
o
r
(
)


n
e
x
t
(
p
)
 
 
 
 
 
#
 
A
d
v
a
n
c
e
 
t
o
 
t
h
e
 
y
i
e
l
d
 
(
r
e
a
d
y
 
t
o
 
r
e
c
e
i
v
e
)


p
.
s
e
n
d
(
'
H
e
l
l
o
'
)


p
.
s
e
n
d
(
'
W
o
r
l
d
'
)


p
.
c
l
o
s
e
(
)



### 讨论


actor模式的魅力就在于它的简单性。
实际上，这里仅仅只有一个核心操作 send() .
甚至，对于在基于actor系统中的“消息”的泛化概念可以已多种方式被扩展。
例如，你可以以元组形式传递标签消息，让actor执行不同的操作，如下：

In [ ]:
c
l
a
s
s
 
T
a
g
g
e
d
A
c
t
o
r
(
A
c
t
o
r
)
:


 
 
 
 
d
e
f
 
r
u
n
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
 
 
 
 
 
t
a
g
,
 
*
p
a
y
l
o
a
d
 
=
 
s
e
l
f
.
r
e
c
v
(
)


 
 
 
 
 
 
 
 
 
 
 
 
 
g
e
t
a
t
t
r
(
s
e
l
f
,
'
d
o
_
'
+
t
a
g
)
(
*
p
a
y
l
o
a
d
)




 
 
 
 
#
 
M
e
t
h
o
d
s
 
c
o
r
r
e
p
o
n
d
i
n
g
 
t
o
 
d
i
f
f
e
r
e
n
t
 
m
e
s
s
a
g
e
 
t
a
g
s


 
 
 
 
d
e
f
 
d
o
_
A
(
s
e
l
f
,
 
x
)
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
R
u
n
n
i
n
g
 
A
'
,
 
x
)




 
 
 
 
d
e
f
 
d
o
_
B
(
s
e
l
f
,
 
x
,
 
y
)
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
R
u
n
n
i
n
g
 
B
'
,
 
x
,
 
y
)




#
 
E
x
a
m
p
l
e


a
 
=
 
T
a
g
g
e
d
A
c
t
o
r
(
)


a
.
s
t
a
r
t
(
)


a
.
s
e
n
d
(
(
'
A
'
,
 
1
)
)
 
 
 
 
 
 
#
 
I
n
v
o
k
e
s
 
d
o
_
A
(
1
)


a
.
s
e
n
d
(
(
'
B
'
,
 
2
,
 
3
)
)
 
 
 
#
 
I
n
v
o
k
e
s
 
d
o
_
B
(
2
,
3
)



作为另外一个例子，下面的actor允许在一个工作者中运行任意的函数，
并且通过一个特殊的Result对象返回结果：

In [ ]:
f
r
o
m
 
t
h
r
e
a
d
i
n
g
 
i
m
p
o
r
t
 
E
v
e
n
t


c
l
a
s
s
 
R
e
s
u
l
t
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
e
v
t
 
=
 
E
v
e
n
t
(
)


 
 
 
 
 
 
 
 
s
e
l
f
.
_
r
e
s
u
l
t
 
=
 
N
o
n
e




 
 
 
 
d
e
f
 
s
e
t
_
r
e
s
u
l
t
(
s
e
l
f
,
 
v
a
l
u
e
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
r
e
s
u
l
t
 
=
 
v
a
l
u
e




 
 
 
 
 
 
 
 
s
e
l
f
.
_
e
v
t
.
s
e
t
(
)




 
 
 
 
d
e
f
 
r
e
s
u
l
t
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
e
v
t
.
w
a
i
t
(
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
s
e
l
f
.
_
r
e
s
u
l
t




c
l
a
s
s
 
W
o
r
k
e
r
(
A
c
t
o
r
)
:


 
 
 
 
d
e
f
 
s
u
b
m
i
t
(
s
e
l
f
,
 
f
u
n
c
,
 
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)
:


 
 
 
 
 
 
 
 
r
 
=
 
R
e
s
u
l
t
(
)


 
 
 
 
 
 
 
 
s
e
l
f
.
s
e
n
d
(
(
f
u
n
c
,
 
a
r
g
s
,
 
k
w
a
r
g
s
,
 
r
)
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
r




 
 
 
 
d
e
f
 
r
u
n
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
 
 
 
 
f
u
n
c
,
 
a
r
g
s
,
 
k
w
a
r
g
s
,
 
r
 
=
 
s
e
l
f
.
r
e
c
v
(
)


 
 
 
 
 
 
 
 
 
 
 
 
r
.
s
e
t
_
r
e
s
u
l
t
(
f
u
n
c
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)
)




#
 
E
x
a
m
p
l
e
 
u
s
e


w
o
r
k
e
r
 
=
 
W
o
r
k
e
r
(
)


w
o
r
k
e
r
.
s
t
a
r
t
(
)


r
 
=
 
w
o
r
k
e
r
.
s
u
b
m
i
t
(
p
o
w
,
 
2
,
 
3
)


p
r
i
n
t
(
r
.
r
e
s
u
l
t
(
)
)



最后，“发送”一个任务消息的概念可以被扩展到多进程甚至是大型分布式系统中去。
例如，一个类actor对象的 send() 方法可以被编程让它能在一个套接字连接上传输数据
或通过某些消息中间件（比如AMQP、ZMQ等）来发送。

## 12.11 实现消息发布/订阅模型


### 问题


你有一个基于线程通信的程序，想让它们实现发布/订阅模式的消息通信。

### 解决方案


要实现发布/订阅的消息通信模式，
你通常要引入一个单独的“交换机”或“网关”对象作为所有消息的中介。
也就是说，不直接将消息从一个任务发送到另一个，而是将其发送给交换机，
然后由交换机将它发送给一个或多个被关联任务。下面是一个非常简单的交换机实现例子：

In [ ]:
f
r
o
m
 
c
o
l
l
e
c
t
i
o
n
s
 
i
m
p
o
r
t
 
d
e
f
a
u
l
t
d
i
c
t




c
l
a
s
s
 
E
x
c
h
a
n
g
e
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
s
u
b
s
c
r
i
b
e
r
s
 
=
 
s
e
t
(
)




 
 
 
 
d
e
f
 
a
t
t
a
c
h
(
s
e
l
f
,
 
t
a
s
k
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
s
u
b
s
c
r
i
b
e
r
s
.
a
d
d
(
t
a
s
k
)




 
 
 
 
d
e
f
 
d
e
t
a
c
h
(
s
e
l
f
,
 
t
a
s
k
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
s
u
b
s
c
r
i
b
e
r
s
.
r
e
m
o
v
e
(
t
a
s
k
)




 
 
 
 
d
e
f
 
s
e
n
d
(
s
e
l
f
,
 
m
s
g
)
:


 
 
 
 
 
 
 
 
f
o
r
 
s
u
b
s
c
r
i
b
e
r
 
i
n
 
s
e
l
f
.
_
s
u
b
s
c
r
i
b
e
r
s
:


 
 
 
 
 
 
 
 
 
 
 
 
s
u
b
s
c
r
i
b
e
r
.
s
e
n
d
(
m
s
g
)




#
 
D
i
c
t
i
o
n
a
r
y
 
o
f
 
a
l
l
 
c
r
e
a
t
e
d
 
e
x
c
h
a
n
g
e
s


_
e
x
c
h
a
n
g
e
s
 
=
 
d
e
f
a
u
l
t
d
i
c
t
(
E
x
c
h
a
n
g
e
)




#
 
R
e
t
u
r
n
 
t
h
e
 
E
x
c
h
a
n
g
e
 
i
n
s
t
a
n
c
e
 
a
s
s
o
c
i
a
t
e
d
 
w
i
t
h
 
a
 
g
i
v
e
n
 
n
a
m
e


d
e
f
 
g
e
t
_
e
x
c
h
a
n
g
e
(
n
a
m
e
)
:


 
 
 
 
r
e
t
u
r
n
 
_
e
x
c
h
a
n
g
e
s
[
n
a
m
e
]



一个交换机就是一个普通对象，负责维护一个活跃的订阅者集合，并为绑定、解绑和发送消息提供相应的方法。
每个交换机通过一个名称定位，get_exchange() 通过给定一个名称返回相应的 Exchange 实例。

下面是一个简单例子，演示了如何使用一个交换机：

In [ ]:
#
 
E
x
a
m
p
l
e
 
o
f
 
a
 
t
a
s
k
.
 
 
A
n
y
 
o
b
j
e
c
t
 
w
i
t
h
 
a
 
s
e
n
d
(
)
 
m
e
t
h
o
d




c
l
a
s
s
 
T
a
s
k
:


 
 
 
 
.
.
.


 
 
 
 
d
e
f
 
s
e
n
d
(
s
e
l
f
,
 
m
s
g
)
:


 
 
 
 
 
 
 
 
.
.
.




t
a
s
k
_
a
 
=
 
T
a
s
k
(
)


t
a
s
k
_
b
 
=
 
T
a
s
k
(
)




#
 
E
x
a
m
p
l
e
 
o
f
 
g
e
t
t
i
n
g
 
a
n
 
e
x
c
h
a
n
g
e


e
x
c
 
=
 
g
e
t
_
e
x
c
h
a
n
g
e
(
'
n
a
m
e
'
)




#
 
E
x
a
m
p
l
e
s
 
o
f
 
s
u
b
s
c
r
i
b
i
n
g
 
t
a
s
k
s
 
t
o
 
i
t


e
x
c
.
a
t
t
a
c
h
(
t
a
s
k
_
a
)


e
x
c
.
a
t
t
a
c
h
(
t
a
s
k
_
b
)




#
 
E
x
a
m
p
l
e
 
o
f
 
s
e
n
d
i
n
g
 
m
e
s
s
a
g
e
s


e
x
c
.
s
e
n
d
(
'
m
s
g
1
'
)


e
x
c
.
s
e
n
d
(
'
m
s
g
2
'
)




#
 
E
x
a
m
p
l
e
 
o
f
 
u
n
s
u
b
s
c
r
i
b
i
n
g


e
x
c
.
d
e
t
a
c
h
(
t
a
s
k
_
a
)


e
x
c
.
d
e
t
a
c
h
(
t
a
s
k
_
b
)



尽管对于这个问题有很多的变种，不过万变不离其宗。
消息会被发送给一个交换机，然后交换机会将它们发送给被绑定的订阅者。

### 讨论


通过队列发送消息的任务或线程的模式很容易被实现并且也非常普遍。
不过，使用发布/订阅模式的好处更加明显。

首先，使用一个交换机可以简化大部分涉及到线程通信的工作。
无需去写通过多进程模块来操作多个线程，你只需要使用这个交换机来连接它们。
某种程度上，这个就跟日志模块的工作原理类似。
实际上，它可以轻松的解耦程序中多个任务。

其次，交换机广播消息给多个订阅者的能力带来了一个全新的通信模式。
例如，你可以使用多任务系统、广播或扇出。
你还可以通过以普通订阅者身份绑定来构建调试和诊断工具。
例如，下面是一个简单的诊断类，可以显示被发送的消息：

In [ ]:
c
l
a
s
s
 
D
i
s
p
l
a
y
M
e
s
s
a
g
e
s
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
c
o
u
n
t
 
=
 
0


 
 
 
 
d
e
f
 
s
e
n
d
(
s
e
l
f
,
 
m
s
g
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
c
o
u
n
t
 
+
=
 
1


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
m
s
g
[
{
}
]
:
 
{
!
r
}
'
.
f
o
r
m
a
t
(
s
e
l
f
.
c
o
u
n
t
,
 
m
s
g
)
)




e
x
c
 
=
 
g
e
t
_
e
x
c
h
a
n
g
e
(
'
n
a
m
e
'
)


d
 
=
 
D
i
s
p
l
a
y
M
e
s
s
a
g
e
s
(
)


e
x
c
.
a
t
t
a
c
h
(
d
)



最后，该实现的一个重要特点是它能兼容多个“task-like”对象。
例如，消息接受者可以是actor（12.10小节介绍）、协程、网络连接或任何实现了正确的 send() 方法的东西。

关于交换机的一个可能问题是对于订阅者的正确绑定和解绑。
为了正确的管理资源，每一个绑定的订阅者必须最终要解绑。
在代码中通常会是像下面这样的模式：

In [ ]:
e
x
c
 
=
 
g
e
t
_
e
x
c
h
a
n
g
e
(
'
n
a
m
e
'
)


e
x
c
.
a
t
t
a
c
h
(
s
o
m
e
_
t
a
s
k
)


t
r
y
:


 
 
 
 
.
.
.


f
i
n
a
l
l
y
:


 
 
 
 
e
x
c
.
d
e
t
a
c
h
(
s
o
m
e
_
t
a
s
k
)



某种意义上，这个和使用文件、锁和类似对象很像。
通常很容易会忘记最后的 detach() 步骤。
为了简化这个，你可以考虑使用上下文管理器协议。
例如，在交换机对象上增加一个 subscribe() 方法，如下：

In [ ]:
f
r
o
m
 
c
o
n
t
e
x
t
l
i
b
 
i
m
p
o
r
t
 
c
o
n
t
e
x
t
m
a
n
a
g
e
r


f
r
o
m
 
c
o
l
l
e
c
t
i
o
n
s
 
i
m
p
o
r
t
 
d
e
f
a
u
l
t
d
i
c
t




c
l
a
s
s
 
E
x
c
h
a
n
g
e
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
s
u
b
s
c
r
i
b
e
r
s
 
=
 
s
e
t
(
)




 
 
 
 
d
e
f
 
a
t
t
a
c
h
(
s
e
l
f
,
 
t
a
s
k
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
s
u
b
s
c
r
i
b
e
r
s
.
a
d
d
(
t
a
s
k
)




 
 
 
 
d
e
f
 
d
e
t
a
c
h
(
s
e
l
f
,
 
t
a
s
k
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
s
u
b
s
c
r
i
b
e
r
s
.
r
e
m
o
v
e
(
t
a
s
k
)




 
 
 
 
@
c
o
n
t
e
x
t
m
a
n
a
g
e
r


 
 
 
 
d
e
f
 
s
u
b
s
c
r
i
b
e
(
s
e
l
f
,
 
*
t
a
s
k
s
)
:


 
 
 
 
 
 
 
 
f
o
r
 
t
a
s
k
 
i
n
 
t
a
s
k
s
:


 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
a
t
t
a
c
h
(
t
a
s
k
)


 
 
 
 
 
 
 
 
t
r
y
:


 
 
 
 
 
 
 
 
 
 
 
 
y
i
e
l
d


 
 
 
 
 
 
 
 
f
i
n
a
l
l
y
:


 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
t
a
s
k
 
i
n
 
t
a
s
k
s
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
d
e
t
a
c
h
(
t
a
s
k
)




 
 
 
 
d
e
f
 
s
e
n
d
(
s
e
l
f
,
 
m
s
g
)
:


 
 
 
 
 
 
 
 
f
o
r
 
s
u
b
s
c
r
i
b
e
r
 
i
n
 
s
e
l
f
.
_
s
u
b
s
c
r
i
b
e
r
s
:


 
 
 
 
 
 
 
 
 
 
 
 
s
u
b
s
c
r
i
b
e
r
.
s
e
n
d
(
m
s
g
)




#
 
D
i
c
t
i
o
n
a
r
y
 
o
f
 
a
l
l
 
c
r
e
a
t
e
d
 
e
x
c
h
a
n
g
e
s


_
e
x
c
h
a
n
g
e
s
 
=
 
d
e
f
a
u
l
t
d
i
c
t
(
E
x
c
h
a
n
g
e
)




#
 
R
e
t
u
r
n
 
t
h
e
 
E
x
c
h
a
n
g
e
 
i
n
s
t
a
n
c
e
 
a
s
s
o
c
i
a
t
e
d
 
w
i
t
h
 
a
 
g
i
v
e
n
 
n
a
m
e


d
e
f
 
g
e
t
_
e
x
c
h
a
n
g
e
(
n
a
m
e
)
:


 
 
 
 
r
e
t
u
r
n
 
_
e
x
c
h
a
n
g
e
s
[
n
a
m
e
]




#
 
E
x
a
m
p
l
e
 
o
f
 
u
s
i
n
g
 
t
h
e
 
s
u
b
s
c
r
i
b
e
(
)
 
m
e
t
h
o
d


e
x
c
 
=
 
g
e
t
_
e
x
c
h
a
n
g
e
(
'
n
a
m
e
'
)


w
i
t
h
 
e
x
c
.
s
u
b
s
c
r
i
b
e
(
t
a
s
k
_
a
,
 
t
a
s
k
_
b
)
:


 
 
 
 
 
.
.
.


 
 
 
 
 
e
x
c
.
s
e
n
d
(
'
m
s
g
1
'
)


 
 
 
 
 
e
x
c
.
s
e
n
d
(
'
m
s
g
2
'
)


 
 
 
 
 
.
.
.




#
 
t
a
s
k
_
a
 
a
n
d
 
t
a
s
k
_
b
 
d
e
t
a
c
h
e
d
 
h
e
r
e



最后还应该注意的是关于交换机的思想有很多种的扩展实现。
例如，交换机可以实现一整个消息通道集合或提供交换机名称的模式匹配规则。
交换机还可以被扩展到分布式计算程序中（比如，将消息路由到不同机器上面的任务中去）。

## 12.12 使用生成器代替线程


### 问题


你想使用生成器（协程）替代系统线程来实现并发。这个有时又被称为用户级线程或绿色线程。

### 解决方案


要使用生成器实现自己的并发，你首先要对生成器函数和 yield 语句有深刻理解。
yield 语句会让一个生成器挂起它的执行，这样就可以编写一个调度器，
将生成器当做某种“任务”并使用任务协作切换来替换它们的执行。
要演示这种思想，考虑下面两个使用简单的 yield 语句的生成器函数：

In [ ]:
#
 
T
w
o
 
s
i
m
p
l
e
 
g
e
n
e
r
a
t
o
r
 
f
u
n
c
t
i
o
n
s


d
e
f
 
c
o
u
n
t
d
o
w
n
(
n
)
:


 
 
 
 
w
h
i
l
e
 
n
 
>
 
0
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
T
-
m
i
n
u
s
'
,
 
n
)


 
 
 
 
 
 
 
 
y
i
e
l
d


 
 
 
 
 
 
 
 
n
 
-
=
 
1


 
 
 
 
p
r
i
n
t
(
'
B
l
a
s
t
o
f
f
!
'
)




d
e
f
 
c
o
u
n
t
u
p
(
n
)
:


 
 
 
 
x
 
=
 
0


 
 
 
 
w
h
i
l
e
 
x
 
<
 
n
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
C
o
u
n
t
i
n
g
 
u
p
'
,
 
x
)


 
 
 
 
 
 
 
 
y
i
e
l
d


 
 
 
 
 
 
 
 
x
 
+
=
 
1



这些函数在内部使用yield语句，下面是一个实现了简单任务调度器的代码：

In [ ]:
f
r
o
m
 
c
o
l
l
e
c
t
i
o
n
s
 
i
m
p
o
r
t
 
d
e
q
u
e




c
l
a
s
s
 
T
a
s
k
S
c
h
e
d
u
l
e
r
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
t
a
s
k
_
q
u
e
u
e
 
=
 
d
e
q
u
e
(
)




 
 
 
 
d
e
f
 
n
e
w
_
t
a
s
k
(
s
e
l
f
,
 
t
a
s
k
)
:


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
A
d
m
i
t
 
a
 
n
e
w
l
y
 
s
t
a
r
t
e
d
 
t
a
s
k
 
t
o
 
t
h
e
 
s
c
h
e
d
u
l
e
r




 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
s
e
l
f
.
_
t
a
s
k
_
q
u
e
u
e
.
a
p
p
e
n
d
(
t
a
s
k
)




 
 
 
 
d
e
f
 
r
u
n
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
R
u
n
 
u
n
t
i
l
 
t
h
e
r
e
 
a
r
e
 
n
o
 
m
o
r
e
 
t
a
s
k
s


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
w
h
i
l
e
 
s
e
l
f
.
_
t
a
s
k
_
q
u
e
u
e
:


 
 
 
 
 
 
 
 
 
 
 
 
t
a
s
k
 
=
 
s
e
l
f
.
_
t
a
s
k
_
q
u
e
u
e
.
p
o
p
l
e
f
t
(
)


 
 
 
 
 
 
 
 
 
 
 
 
t
r
y
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
R
u
n
 
u
n
t
i
l
 
t
h
e
 
n
e
x
t
 
y
i
e
l
d
 
s
t
a
t
e
m
e
n
t


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
n
e
x
t
(
t
a
s
k
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
t
a
s
k
_
q
u
e
u
e
.
a
p
p
e
n
d
(
t
a
s
k
)


 
 
 
 
 
 
 
 
 
 
 
 
e
x
c
e
p
t
 
S
t
o
p
I
t
e
r
a
t
i
o
n
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
G
e
n
e
r
a
t
o
r
 
i
s
 
n
o
 
l
o
n
g
e
r
 
e
x
e
c
u
t
i
n
g


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
a
s
s




#
 
E
x
a
m
p
l
e
 
u
s
e


s
c
h
e
d
 
=
 
T
a
s
k
S
c
h
e
d
u
l
e
r
(
)


s
c
h
e
d
.
n
e
w
_
t
a
s
k
(
c
o
u
n
t
d
o
w
n
(
1
0
)
)


s
c
h
e
d
.
n
e
w
_
t
a
s
k
(
c
o
u
n
t
d
o
w
n
(
5
)
)


s
c
h
e
d
.
n
e
w
_
t
a
s
k
(
c
o
u
n
t
u
p
(
1
5
)
)


s
c
h
e
d
.
r
u
n
(
)



TaskScheduler 类在一个循环中运行生成器集合——每个都运行到碰到yield语句为止。
运行这个例子，输出如下：

In [ ]:
T
-
m
i
n
u
s
 
1
0


T
-
m
i
n
u
s
 
5


C
o
u
n
t
i
n
g
 
u
p
 
0


T
-
m
i
n
u
s
 
9


T
-
m
i
n
u
s
 
4


C
o
u
n
t
i
n
g
 
u
p
 
1


T
-
m
i
n
u
s
 
8


T
-
m
i
n
u
s
 
3


C
o
u
n
t
i
n
g
 
u
p
 
2


T
-
m
i
n
u
s
 
7


T
-
m
i
n
u
s
 
2


.
.
.



到此为止，我们实际上已经实现了一个“操作系统”的最小核心部分。
生成器函数就是认为，而yield语句是任务挂起的信号。
调度器循环检查任务列表直到没有任务要执行为止。

实际上，你可能想要使用生成器来实现简单的并发。
那么，在实现actor或网络服务器的时候你可以使用生成器来替代线程的使用。

下面的代码演示了使用生成器来实现一个不依赖线程的actor：

In [ ]:
f
r
o
m
 
c
o
l
l
e
c
t
i
o
n
s
 
i
m
p
o
r
t
 
d
e
q
u
e




c
l
a
s
s
 
A
c
t
o
r
S
c
h
e
d
u
l
e
r
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
a
c
t
o
r
s
 
=
 
{
 
}
 
 
 
 
 
 
 
 
 
 
#
 
M
a
p
p
i
n
g
 
o
f
 
n
a
m
e
s
 
t
o
 
a
c
t
o
r
s


 
 
 
 
 
 
 
 
s
e
l
f
.
_
m
s
g
_
q
u
e
u
e
 
=
 
d
e
q
u
e
(
)
 
 
 
#
 
M
e
s
s
a
g
e
 
q
u
e
u
e




 
 
 
 
d
e
f
 
n
e
w
_
a
c
t
o
r
(
s
e
l
f
,
 
n
a
m
e
,
 
a
c
t
o
r
)
:


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
A
d
m
i
t
 
a
 
n
e
w
l
y
 
s
t
a
r
t
e
d
 
a
c
t
o
r
 
t
o
 
t
h
e
 
s
c
h
e
d
u
l
e
r
 
a
n
d
 
g
i
v
e
 
i
t
 
a
 
n
a
m
e


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
s
e
l
f
.
_
m
s
g
_
q
u
e
u
e
.
a
p
p
e
n
d
(
(
a
c
t
o
r
,
N
o
n
e
)
)


 
 
 
 
 
 
 
 
s
e
l
f
.
_
a
c
t
o
r
s
[
n
a
m
e
]
 
=
 
a
c
t
o
r




 
 
 
 
d
e
f
 
s
e
n
d
(
s
e
l
f
,
 
n
a
m
e
,
 
m
s
g
)
:


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
S
e
n
d
 
a
 
m
e
s
s
a
g
e
 
t
o
 
a
 
n
a
m
e
d
 
a
c
t
o
r


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
a
c
t
o
r
 
=
 
s
e
l
f
.
_
a
c
t
o
r
s
.
g
e
t
(
n
a
m
e
)


 
 
 
 
 
 
 
 
i
f
 
a
c
t
o
r
:


 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
m
s
g
_
q
u
e
u
e
.
a
p
p
e
n
d
(
(
a
c
t
o
r
,
m
s
g
)
)




 
 
 
 
d
e
f
 
r
u
n
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
R
u
n
 
a
s
 
l
o
n
g
 
a
s
 
t
h
e
r
e
 
a
r
e
 
p
e
n
d
i
n
g
 
m
e
s
s
a
g
e
s
.


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
w
h
i
l
e
 
s
e
l
f
.
_
m
s
g
_
q
u
e
u
e
:


 
 
 
 
 
 
 
 
 
 
 
 
a
c
t
o
r
,
 
m
s
g
 
=
 
s
e
l
f
.
_
m
s
g
_
q
u
e
u
e
.
p
o
p
l
e
f
t
(
)


 
 
 
 
 
 
 
 
 
 
 
 
t
r
y
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
a
c
t
o
r
.
s
e
n
d
(
m
s
g
)


 
 
 
 
 
 
 
 
 
 
 
 
e
x
c
e
p
t
 
S
t
o
p
I
t
e
r
a
t
i
o
n
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
a
s
s




#
 
E
x
a
m
p
l
e
 
u
s
e


i
f
 
_
_
n
a
m
e
_
_
 
=
=
 
'
_
_
m
a
i
n
_
_
'
:


 
 
 
 
d
e
f
 
p
r
i
n
t
e
r
(
)
:


 
 
 
 
 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
 
 
 
 
m
s
g
 
=
 
y
i
e
l
d


 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
G
o
t
:
'
,
 
m
s
g
)




 
 
 
 
d
e
f
 
c
o
u
n
t
e
r
(
s
c
h
e
d
)
:


 
 
 
 
 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
 
 
 
 
#
 
R
e
c
e
i
v
e
 
t
h
e
 
c
u
r
r
e
n
t
 
c
o
u
n
t


 
 
 
 
 
 
 
 
 
 
 
 
n
 
=
 
y
i
e
l
d


 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
n
 
=
=
 
0
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
b
r
e
a
k


 
 
 
 
 
 
 
 
 
 
 
 
#
 
S
e
n
d
 
t
o
 
t
h
e
 
p
r
i
n
t
e
r
 
t
a
s
k


 
 
 
 
 
 
 
 
 
 
 
 
s
c
h
e
d
.
s
e
n
d
(
'
p
r
i
n
t
e
r
'
,
 
n
)


 
 
 
 
 
 
 
 
 
 
 
 
#
 
S
e
n
d
 
t
h
e
 
n
e
x
t
 
c
o
u
n
t
 
t
o
 
t
h
e
 
c
o
u
n
t
e
r
 
t
a
s
k
 
(
r
e
c
u
r
s
i
v
e
)




 
 
 
 
 
 
 
 
 
 
 
 
s
c
h
e
d
.
s
e
n
d
(
'
c
o
u
n
t
e
r
'
,
 
n
-
1
)




 
 
 
 
s
c
h
e
d
 
=
 
A
c
t
o
r
S
c
h
e
d
u
l
e
r
(
)


 
 
 
 
#
 
C
r
e
a
t
e
 
t
h
e
 
i
n
i
t
i
a
l
 
a
c
t
o
r
s


 
 
 
 
s
c
h
e
d
.
n
e
w
_
a
c
t
o
r
(
'
p
r
i
n
t
e
r
'
,
 
p
r
i
n
t
e
r
(
)
)


 
 
 
 
s
c
h
e
d
.
n
e
w
_
a
c
t
o
r
(
'
c
o
u
n
t
e
r
'
,
 
c
o
u
n
t
e
r
(
s
c
h
e
d
)
)




 
 
 
 
#
 
S
e
n
d
 
a
n
 
i
n
i
t
i
a
l
 
m
e
s
s
a
g
e
 
t
o
 
t
h
e
 
c
o
u
n
t
e
r
 
t
o
 
i
n
i
t
i
a
t
e


 
 
 
 
s
c
h
e
d
.
s
e
n
d
(
'
c
o
u
n
t
e
r
'
,
 
1
0
0
0
0
)


 
 
 
 
s
c
h
e
d
.
r
u
n
(
)



完全弄懂这段代码需要更深入的学习，但是关键点在于收集消息的队列。
本质上，调度器在有需要发送的消息时会一直运行着。
计数生成器会给自己发送消息并在一个递归循环中结束。

下面是一个更加高级的例子，演示了使用生成器来实现一个并发网络应用程序：

In [ ]:
f
r
o
m
 
c
o
l
l
e
c
t
i
o
n
s
 
i
m
p
o
r
t
 
d
e
q
u
e


f
r
o
m
 
s
e
l
e
c
t
 
i
m
p
o
r
t
 
s
e
l
e
c
t




#
 
T
h
i
s
 
c
l
a
s
s
 
r
e
p
r
e
s
e
n
t
s
 
a
 
g
e
n
e
r
i
c
 
y
i
e
l
d
 
e
v
e
n
t
 
i
n
 
t
h
e
 
s
c
h
e
d
u
l
e
r


c
l
a
s
s
 
Y
i
e
l
d
E
v
e
n
t
:


 
 
 
 
d
e
f
 
h
a
n
d
l
e
_
y
i
e
l
d
(
s
e
l
f
,
 
s
c
h
e
d
,
 
t
a
s
k
)
:


 
 
 
 
 
 
 
 
p
a
s
s


 
 
 
 
d
e
f
 
h
a
n
d
l
e
_
r
e
s
u
m
e
(
s
e
l
f
,
 
s
c
h
e
d
,
 
t
a
s
k
)
:


 
 
 
 
 
 
 
 
p
a
s
s




#
 
T
a
s
k
 
S
c
h
e
d
u
l
e
r


c
l
a
s
s
 
S
c
h
e
d
u
l
e
r
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
n
u
m
t
a
s
k
s
 
=
 
0
 
 
 
 
 
 
 
#
 
T
o
t
a
l
 
n
u
m
 
o
f
 
t
a
s
k
s


 
 
 
 
 
 
 
 
s
e
l
f
.
_
r
e
a
d
y
 
=
 
d
e
q
u
e
(
)
 
 
 
 
#
 
T
a
s
k
s
 
r
e
a
d
y
 
t
o
 
r
u
n


 
 
 
 
 
 
 
 
s
e
l
f
.
_
r
e
a
d
_
w
a
i
t
i
n
g
 
=
 
{
}
 
 
#
 
T
a
s
k
s
 
w
a
i
t
i
n
g
 
t
o
 
r
e
a
d


 
 
 
 
 
 
 
 
s
e
l
f
.
_
w
r
i
t
e
_
w
a
i
t
i
n
g
 
=
 
{
}
 
#
 
T
a
s
k
s
 
w
a
i
t
i
n
g
 
t
o
 
w
r
i
t
e




 
 
 
 
#
 
P
o
l
l
 
f
o
r
 
I
/
O
 
e
v
e
n
t
s
 
a
n
d
 
r
e
s
t
a
r
t
 
w
a
i
t
i
n
g
 
t
a
s
k
s


 
 
 
 
d
e
f
 
_
i
o
p
o
l
l
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
r
s
e
t
,
w
s
e
t
,
e
s
e
t
 
=
 
s
e
l
e
c
t
(
s
e
l
f
.
_
r
e
a
d
_
w
a
i
t
i
n
g
,


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
w
r
i
t
e
_
w
a
i
t
i
n
g
,
[
]
)


 
 
 
 
 
 
 
 
f
o
r
 
r
 
i
n
 
r
s
e
t
:


 
 
 
 
 
 
 
 
 
 
 
 
e
v
t
,
 
t
a
s
k
 
=
 
s
e
l
f
.
_
r
e
a
d
_
w
a
i
t
i
n
g
.
p
o
p
(
r
)


 
 
 
 
 
 
 
 
 
 
 
 
e
v
t
.
h
a
n
d
l
e
_
r
e
s
u
m
e
(
s
e
l
f
,
 
t
a
s
k
)


 
 
 
 
 
 
 
 
f
o
r
 
w
 
i
n
 
w
s
e
t
:


 
 
 
 
 
 
 
 
 
 
 
 
e
v
t
,
 
t
a
s
k
 
=
 
s
e
l
f
.
_
w
r
i
t
e
_
w
a
i
t
i
n
g
.
p
o
p
(
w
)


 
 
 
 
 
 
 
 
 
 
 
 
e
v
t
.
h
a
n
d
l
e
_
r
e
s
u
m
e
(
s
e
l
f
,
 
t
a
s
k
)




 
 
 
 
d
e
f
 
n
e
w
(
s
e
l
f
,
t
a
s
k
)
:


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
A
d
d
 
a
 
n
e
w
l
y
 
s
t
a
r
t
e
d
 
t
a
s
k
 
t
o
 
t
h
e
 
s
c
h
e
d
u
l
e
r


 
 
 
 
 
 
 
 
'
'
'




 
 
 
 
 
 
 
 
s
e
l
f
.
_
r
e
a
d
y
.
a
p
p
e
n
d
(
(
t
a
s
k
,
 
N
o
n
e
)
)


 
 
 
 
 
 
 
 
s
e
l
f
.
_
n
u
m
t
a
s
k
s
 
+
=
 
1




 
 
 
 
d
e
f
 
a
d
d
_
r
e
a
d
y
(
s
e
l
f
,
 
t
a
s
k
,
 
m
s
g
=
N
o
n
e
)
:


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
A
p
p
e
n
d
 
a
n
 
a
l
r
e
a
d
y
 
s
t
a
r
t
e
d
 
t
a
s
k
 
t
o
 
t
h
e
 
r
e
a
d
y
 
q
u
e
u
e
.


 
 
 
 
 
 
 
 
m
s
g
 
i
s
 
w
h
a
t
 
t
o
 
s
e
n
d
 
i
n
t
o
 
t
h
e
 
t
a
s
k
 
w
h
e
n
 
i
t
 
r
e
s
u
m
e
s
.


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
s
e
l
f
.
_
r
e
a
d
y
.
a
p
p
e
n
d
(
(
t
a
s
k
,
 
m
s
g
)
)




 
 
 
 
#
 
A
d
d
 
a
 
t
a
s
k
 
t
o
 
t
h
e
 
r
e
a
d
i
n
g
 
s
e
t


 
 
 
 
d
e
f
 
_
r
e
a
d
_
w
a
i
t
(
s
e
l
f
,
 
f
i
l
e
n
o
,
 
e
v
t
,
 
t
a
s
k
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
r
e
a
d
_
w
a
i
t
i
n
g
[
f
i
l
e
n
o
]
 
=
 
(
e
v
t
,
 
t
a
s
k
)




 
 
 
 
#
 
A
d
d
 
a
 
t
a
s
k
 
t
o
 
t
h
e
 
w
r
i
t
e
 
s
e
t


 
 
 
 
d
e
f
 
_
w
r
i
t
e
_
w
a
i
t
(
s
e
l
f
,
 
f
i
l
e
n
o
,
 
e
v
t
,
 
t
a
s
k
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
w
r
i
t
e
_
w
a
i
t
i
n
g
[
f
i
l
e
n
o
]
 
=
 
(
e
v
t
,
 
t
a
s
k
)




 
 
 
 
d
e
f
 
r
u
n
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
R
u
n
 
t
h
e
 
t
a
s
k
 
s
c
h
e
d
u
l
e
r
 
u
n
t
i
l
 
t
h
e
r
e
 
a
r
e
 
n
o
 
t
a
s
k
s


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
w
h
i
l
e
 
s
e
l
f
.
_
n
u
m
t
a
s
k
s
:


 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
n
o
t
 
s
e
l
f
.
_
r
e
a
d
y
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
i
o
p
o
l
l
(
)


 
 
 
 
 
 
 
 
 
 
 
 
 
t
a
s
k
,
 
m
s
g
 
=
 
s
e
l
f
.
_
r
e
a
d
y
.
p
o
p
l
e
f
t
(
)


 
 
 
 
 
 
 
 
 
 
 
 
 
t
r
y
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
R
u
n
 
t
h
e
 
c
o
r
o
u
t
i
n
e
 
t
o
 
t
h
e
 
n
e
x
t
 
y
i
e
l
d


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
r
 
=
 
t
a
s
k
.
s
e
n
d
(
m
s
g
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
i
s
i
n
s
t
a
n
c
e
(
r
,
 
Y
i
e
l
d
E
v
e
n
t
)
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
r
.
h
a
n
d
l
e
_
y
i
e
l
d
(
s
e
l
f
,
 
t
a
s
k
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
e
l
s
e
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
r
a
i
s
e
 
R
u
n
t
i
m
e
E
r
r
o
r
(
'
u
n
r
e
c
o
g
n
i
z
e
d
 
y
i
e
l
d
 
e
v
e
n
t
'
)


 
 
 
 
 
 
 
 
 
 
 
 
 
e
x
c
e
p
t
 
S
t
o
p
I
t
e
r
a
t
i
o
n
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
n
u
m
t
a
s
k
s
 
-
=
 
1




#
 
E
x
a
m
p
l
e
 
i
m
p
l
e
m
e
n
t
a
t
i
o
n
 
o
f
 
c
o
r
o
u
t
i
n
e
-
b
a
s
e
d
 
s
o
c
k
e
t
 
I
/
O


c
l
a
s
s
 
R
e
a
d
S
o
c
k
e
t
(
Y
i
e
l
d
E
v
e
n
t
)
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
s
o
c
k
,
 
n
b
y
t
e
s
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
s
o
c
k
 
=
 
s
o
c
k


 
 
 
 
 
 
 
 
s
e
l
f
.
n
b
y
t
e
s
 
=
 
n
b
y
t
e
s


 
 
 
 
d
e
f
 
h
a
n
d
l
e
_
y
i
e
l
d
(
s
e
l
f
,
 
s
c
h
e
d
,
 
t
a
s
k
)
:


 
 
 
 
 
 
 
 
s
c
h
e
d
.
_
r
e
a
d
_
w
a
i
t
(
s
e
l
f
.
s
o
c
k
.
f
i
l
e
n
o
(
)
,
 
s
e
l
f
,
 
t
a
s
k
)


 
 
 
 
d
e
f
 
h
a
n
d
l
e
_
r
e
s
u
m
e
(
s
e
l
f
,
 
s
c
h
e
d
,
 
t
a
s
k
)
:


 
 
 
 
 
 
 
 
d
a
t
a
 
=
 
s
e
l
f
.
s
o
c
k
.
r
e
c
v
(
s
e
l
f
.
n
b
y
t
e
s
)


 
 
 
 
 
 
 
 
s
c
h
e
d
.
a
d
d
_
r
e
a
d
y
(
t
a
s
k
,
 
d
a
t
a
)




c
l
a
s
s
 
W
r
i
t
e
S
o
c
k
e
t
(
Y
i
e
l
d
E
v
e
n
t
)
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
s
o
c
k
,
 
d
a
t
a
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
s
o
c
k
 
=
 
s
o
c
k


 
 
 
 
 
 
 
 
s
e
l
f
.
d
a
t
a
 
=
 
d
a
t
a


 
 
 
 
d
e
f
 
h
a
n
d
l
e
_
y
i
e
l
d
(
s
e
l
f
,
 
s
c
h
e
d
,
 
t
a
s
k
)
:




 
 
 
 
 
 
 
 
s
c
h
e
d
.
_
w
r
i
t
e
_
w
a
i
t
(
s
e
l
f
.
s
o
c
k
.
f
i
l
e
n
o
(
)
,
 
s
e
l
f
,
 
t
a
s
k
)


 
 
 
 
d
e
f
 
h
a
n
d
l
e
_
r
e
s
u
m
e
(
s
e
l
f
,
 
s
c
h
e
d
,
 
t
a
s
k
)
:


 
 
 
 
 
 
 
 
n
s
e
n
t
 
=
 
s
e
l
f
.
s
o
c
k
.
s
e
n
d
(
s
e
l
f
.
d
a
t
a
)


 
 
 
 
 
 
 
 
s
c
h
e
d
.
a
d
d
_
r
e
a
d
y
(
t
a
s
k
,
 
n
s
e
n
t
)




c
l
a
s
s
 
A
c
c
e
p
t
S
o
c
k
e
t
(
Y
i
e
l
d
E
v
e
n
t
)
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
s
o
c
k
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
s
o
c
k
 
=
 
s
o
c
k


 
 
 
 
d
e
f
 
h
a
n
d
l
e
_
y
i
e
l
d
(
s
e
l
f
,
 
s
c
h
e
d
,
 
t
a
s
k
)
:


 
 
 
 
 
 
 
 
s
c
h
e
d
.
_
r
e
a
d
_
w
a
i
t
(
s
e
l
f
.
s
o
c
k
.
f
i
l
e
n
o
(
)
,
 
s
e
l
f
,
 
t
a
s
k
)


 
 
 
 
d
e
f
 
h
a
n
d
l
e
_
r
e
s
u
m
e
(
s
e
l
f
,
 
s
c
h
e
d
,
 
t
a
s
k
)
:


 
 
 
 
 
 
 
 
r
 
=
 
s
e
l
f
.
s
o
c
k
.
a
c
c
e
p
t
(
)


 
 
 
 
 
 
 
 
s
c
h
e
d
.
a
d
d
_
r
e
a
d
y
(
t
a
s
k
,
 
r
)




#
 
W
r
a
p
p
e
r
 
a
r
o
u
n
d
 
a
 
s
o
c
k
e
t
 
o
b
j
e
c
t
 
f
o
r
 
u
s
e
 
w
i
t
h
 
y
i
e
l
d


c
l
a
s
s
 
S
o
c
k
e
t
(
o
b
j
e
c
t
)
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
s
o
c
k
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
s
o
c
k
 
=
 
s
o
c
k


 
 
 
 
d
e
f
 
r
e
c
v
(
s
e
l
f
,
 
m
a
x
b
y
t
e
s
)
:


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
R
e
a
d
S
o
c
k
e
t
(
s
e
l
f
.
_
s
o
c
k
,
 
m
a
x
b
y
t
e
s
)


 
 
 
 
d
e
f
 
s
e
n
d
(
s
e
l
f
,
 
d
a
t
a
)
:


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
W
r
i
t
e
S
o
c
k
e
t
(
s
e
l
f
.
_
s
o
c
k
,
 
d
a
t
a
)


 
 
 
 
d
e
f
 
a
c
c
e
p
t
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
A
c
c
e
p
t
S
o
c
k
e
t
(
s
e
l
f
.
_
s
o
c
k
)


 
 
 
 
d
e
f
 
_
_
g
e
t
a
t
t
r
_
_
(
s
e
l
f
,
 
n
a
m
e
)
:


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
g
e
t
a
t
t
r
(
s
e
l
f
.
_
s
o
c
k
,
 
n
a
m
e
)




i
f
 
_
_
n
a
m
e
_
_
 
=
=
 
'
_
_
m
a
i
n
_
_
'
:


 
 
 
 
f
r
o
m
 
s
o
c
k
e
t
 
i
m
p
o
r
t
 
s
o
c
k
e
t
,
 
A
F
_
I
N
E
T
,
 
S
O
C
K
_
S
T
R
E
A
M


 
 
 
 
i
m
p
o
r
t
 
t
i
m
e




 
 
 
 
#
 
E
x
a
m
p
l
e
 
o
f
 
a
 
f
u
n
c
t
i
o
n
 
i
n
v
o
l
v
i
n
g
 
g
e
n
e
r
a
t
o
r
s
.
 
 
T
h
i
s
 
s
h
o
u
l
d


 
 
 
 
#
 
b
e
 
c
a
l
l
e
d
 
u
s
i
n
g
 
l
i
n
e
 
=
 
y
i
e
l
d
 
f
r
o
m
 
r
e
a
d
l
i
n
e
(
s
o
c
k
)


 
 
 
 
d
e
f
 
r
e
a
d
l
i
n
e
(
s
o
c
k
)
:


 
 
 
 
 
 
 
 
c
h
a
r
s
 
=
 
[
]


 
 
 
 
 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
 
 
 
 
c
 
=
 
y
i
e
l
d
 
s
o
c
k
.
r
e
c
v
(
1
)


 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
n
o
t
 
c
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
b
r
e
a
k


 
 
 
 
 
 
 
 
 
 
 
 
c
h
a
r
s
.
a
p
p
e
n
d
(
c
)


 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
c
 
=
=
 
b
'
\
n
'
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
b
r
e
a
k


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
b
'
'
.
j
o
i
n
(
c
h
a
r
s
)




 
 
 
 
#
 
E
c
h
o
 
s
e
r
v
e
r
 
u
s
i
n
g
 
g
e
n
e
r
a
t
o
r
s


 
 
 
 
c
l
a
s
s
 
E
c
h
o
S
e
r
v
e
r
:


 
 
 
 
 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
a
d
d
r
,
s
c
h
e
d
)
:


 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
s
c
h
e
d
 
=
 
s
c
h
e
d


 
 
 
 
 
 
 
 
 
 
 
 
s
c
h
e
d
.
n
e
w
(
s
e
l
f
.
s
e
r
v
e
r
_
l
o
o
p
(
a
d
d
r
)
)




 
 
 
 
 
 
 
 
d
e
f
 
s
e
r
v
e
r
_
l
o
o
p
(
s
e
l
f
,
a
d
d
r
)
:


 
 
 
 
 
 
 
 
 
 
 
 
s
 
=
 
S
o
c
k
e
t
(
s
o
c
k
e
t
(
A
F
_
I
N
E
T
,
S
O
C
K
_
S
T
R
E
A
M
)
)




 
 
 
 
 
 
 
 
 
 
 
 
s
.
b
i
n
d
(
a
d
d
r
)


 
 
 
 
 
 
 
 
 
 
 
 
s
.
l
i
s
t
e
n
(
5
)


 
 
 
 
 
 
 
 
 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
c
,
a
 
=
 
y
i
e
l
d
 
s
.
a
c
c
e
p
t
(
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
G
o
t
 
c
o
n
n
e
c
t
i
o
n
 
f
r
o
m
 
'
,
 
a
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
s
c
h
e
d
.
n
e
w
(
s
e
l
f
.
c
l
i
e
n
t
_
h
a
n
d
l
e
r
(
S
o
c
k
e
t
(
c
)
)
)




 
 
 
 
 
 
 
 
d
e
f
 
c
l
i
e
n
t
_
h
a
n
d
l
e
r
(
s
e
l
f
,
c
l
i
e
n
t
)
:


 
 
 
 
 
 
 
 
 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
l
i
n
e
 
=
 
y
i
e
l
d
 
f
r
o
m
 
r
e
a
d
l
i
n
e
(
c
l
i
e
n
t
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
n
o
t
 
l
i
n
e
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
b
r
e
a
k


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
l
i
n
e
 
=
 
b
'
G
O
T
:
'
 
+
 
l
i
n
e


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
w
h
i
l
e
 
l
i
n
e
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
n
s
e
n
t
 
=
 
y
i
e
l
d
 
c
l
i
e
n
t
.
s
e
n
d
(
l
i
n
e
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
l
i
n
e
 
=
 
l
i
n
e
[
n
s
e
n
t
:
]


 
 
 
 
 
 
 
 
 
 
 
 
c
l
i
e
n
t
.
c
l
o
s
e
(
)


 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
C
l
i
e
n
t
 
c
l
o
s
e
d
'
)




 
 
 
 
s
c
h
e
d
 
=
 
S
c
h
e
d
u
l
e
r
(
)


 
 
 
 
E
c
h
o
S
e
r
v
e
r
(
(
'
'
,
1
6
0
0
0
)
,
s
c
h
e
d
)


 
 
 
 
s
c
h
e
d
.
r
u
n
(
)



这段代码有点复杂。不过，它实现了一个小型的操作系统。
有一个就绪的任务队列，并且还有因I/O休眠的任务等待区域。
还有很多调度器负责在就绪队列和I/O等待区域之间移动任务。

### 讨论


在构建基于生成器的并发框架时，通常会使用更常见的yield形式：

In [ ]:
d
e
f
 
s
o
m
e
_
g
e
n
e
r
a
t
o
r
(
)
:


 
 
 
 
.
.
.


 
 
 
 
r
e
s
u
l
t
 
=
 
y
i
e
l
d
 
d
a
t
a


 
 
 
 
.
.
.



使用这种形式的yield语句的函数通常被称为“协程”。
通过调度器，yield语句在一个循环中被处理，如下：

In [ ]:
f
 
=
 
s
o
m
e
_
g
e
n
e
r
a
t
o
r
(
)




#
 
I
n
i
t
i
a
l
 
r
e
s
u
l
t
.
 
I
s
 
N
o
n
e
 
t
o
 
s
t
a
r
t
 
s
i
n
c
e
 
n
o
t
h
i
n
g
 
h
a
s
 
b
e
e
n
 
c
o
m
p
u
t
e
d


r
e
s
u
l
t
 
=
 
N
o
n
e


w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
t
r
y
:


 
 
 
 
 
 
 
 
d
a
t
a
 
=
 
f
.
s
e
n
d
(
r
e
s
u
l
t
)


 
 
 
 
 
 
 
 
r
e
s
u
l
t
 
=
 
.
.
.
 
d
o
 
s
o
m
e
 
c
a
l
c
u
l
a
t
i
o
n
 
.
.
.


 
 
 
 
e
x
c
e
p
t
 
S
t
o
p
I
t
e
r
a
t
i
o
n
:


 
 
 
 
 
 
 
 
b
r
e
a
k



这里的逻辑稍微有点复杂。不过，被传给 send() 的值定义了在yield语句醒来时的返回值。
因此，如果一个yield准备在对之前yield数据的回应中返回结果时，会在下一次 send() 操作返回。
如果一个生成器函数刚开始运行，发送一个None值会让它排在第一个yield语句前面。

除了发送值外，还可以在一个生成器上面执行一个 close() 方法。
它会导致在执行yield语句时抛出一个 GeneratorExit 异常，从而终止执行。
如果进一步设计，一个生成器可以捕获这个异常并执行清理操作。
同样还可以使用生成器的 throw() 方法在yield语句执行时生成一个任意的执行指令。
一个任务调度器可利用它来在运行的生成器中处理错误。

最后一个例子中使用的 yield from 语句被用来实现协程，可以被其它生成器作为子程序或过程来调用。
本质上就是将控制权透明的传输给新的函数。
不像普通的生成器，一个使用 yield from 被调用的函数可以返回一个作为 yield from 语句结果的值。
关于 yield from 的更多信息可以在 PEP 380 中找到。

最后，如果使用生成器编程，要提醒你的是它还是有很多缺点的。
特别是，你得不到任何线程可以提供的好处。例如，如果你执行CPU依赖或I/O阻塞程序，
它会将整个任务挂起知道操作完成。为了解决这个问题，
你只能选择将操作委派给另外一个可以独立运行的线程或进程。
另外一个限制是大部分Python库并不能很好的兼容基于生成器的线程。
如果你选择这个方案，你会发现你需要自己改写很多标准库函数。
作为本节提到的协程和相关技术的一个基础背景，可以查看 PEP 342
和 “协程和并发的一门有趣课程”

PEP 3156 同样有一个关于使用协程的异步I/O模型。
特别的，你不可能自己去实现一个底层的协程调度器。
不过，关于协程的思想是很多流行库的基础，
包括 gevent,
greenlet,
Stackless Python 以及其他类似工程。

## 12.13 多个线程队列轮询


### 问题


你有一个线程队列集合，想为到来的元素轮询它们，
就跟你为一个客户端请求去轮询一个网络连接集合的方式一样。

### 解决方案


对于轮询问题的一个常见解决方案中有个很少有人知道的技巧，包含了一个隐藏的回路网络连接。
本质上讲其思想就是：对于每个你想要轮询的队列，你创建一对连接的套接字。
然后你在其中一个套接字上面编写代码来标识存在的数据，
另外一个套接字被传给 select() 或类似的一个轮询数据到达的函数。下面的例子演示了这个思想：

In [ ]:
i
m
p
o
r
t
 
q
u
e
u
e


i
m
p
o
r
t
 
s
o
c
k
e
t


i
m
p
o
r
t
 
o
s




c
l
a
s
s
 
P
o
l
l
a
b
l
e
Q
u
e
u
e
(
q
u
e
u
e
.
Q
u
e
u
e
)
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
s
u
p
e
r
(
)
.
_
_
i
n
i
t
_
_
(
)


 
 
 
 
 
 
 
 
#
 
C
r
e
a
t
e
 
a
 
p
a
i
r
 
o
f
 
c
o
n
n
e
c
t
e
d
 
s
o
c
k
e
t
s


 
 
 
 
 
 
 
 
i
f
 
o
s
.
n
a
m
e
 
=
=
 
'
p
o
s
i
x
'
:


 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
p
u
t
s
o
c
k
e
t
,
 
s
e
l
f
.
_
g
e
t
s
o
c
k
e
t
 
=
 
s
o
c
k
e
t
.
s
o
c
k
e
t
p
a
i
r
(
)


 
 
 
 
 
 
 
 
e
l
s
e
:


 
 
 
 
 
 
 
 
 
 
 
 
#
 
C
o
m
p
a
t
i
b
i
l
i
t
y
 
o
n
 
n
o
n
-
P
O
S
I
X
 
s
y
s
t
e
m
s


 
 
 
 
 
 
 
 
 
 
 
 
s
e
r
v
e
r
 
=
 
s
o
c
k
e
t
.
s
o
c
k
e
t
(
s
o
c
k
e
t
.
A
F
_
I
N
E
T
,
 
s
o
c
k
e
t
.
S
O
C
K
_
S
T
R
E
A
M
)


 
 
 
 
 
 
 
 
 
 
 
 
s
e
r
v
e
r
.
b
i
n
d
(
(
'
1
2
7
.
0
.
0
.
1
'
,
 
0
)
)


 
 
 
 
 
 
 
 
 
 
 
 
s
e
r
v
e
r
.
l
i
s
t
e
n
(
1
)


 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
p
u
t
s
o
c
k
e
t
 
=
 
s
o
c
k
e
t
.
s
o
c
k
e
t
(
s
o
c
k
e
t
.
A
F
_
I
N
E
T
,
 
s
o
c
k
e
t
.
S
O
C
K
_
S
T
R
E
A
M
)


 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
p
u
t
s
o
c
k
e
t
.
c
o
n
n
e
c
t
(
s
e
r
v
e
r
.
g
e
t
s
o
c
k
n
a
m
e
(
)
)


 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
g
e
t
s
o
c
k
e
t
,
 
_
 
=
 
s
e
r
v
e
r
.
a
c
c
e
p
t
(
)


 
 
 
 
 
 
 
 
 
 
 
 
s
e
r
v
e
r
.
c
l
o
s
e
(
)




 
 
 
 
d
e
f
 
f
i
l
e
n
o
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
s
e
l
f
.
_
g
e
t
s
o
c
k
e
t
.
f
i
l
e
n
o
(
)




 
 
 
 
d
e
f
 
p
u
t
(
s
e
l
f
,
 
i
t
e
m
)
:


 
 
 
 
 
 
 
 
s
u
p
e
r
(
)
.
p
u
t
(
i
t
e
m
)


 
 
 
 
 
 
 
 
s
e
l
f
.
_
p
u
t
s
o
c
k
e
t
.
s
e
n
d
(
b
'
x
'
)




 
 
 
 
d
e
f
 
g
e
t
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
g
e
t
s
o
c
k
e
t
.
r
e
c
v
(
1
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
s
u
p
e
r
(
)
.
g
e
t
(
)



在这个代码中，一个新的 Queue 实例类型被定义，底层是一个被连接套接字对。
在Unix机器上的 socketpair() 函数能轻松的创建这样的套接字。
在Windows上面，你必须使用类似代码来模拟它。
然后定义普通的 get() 和 put() 方法在这些套接字上面来执行I/O操作。
put() 方法再将数据放入队列后会写一个单字节到某个套接字中去。
而 get() 方法在从队列中移除一个元素时会从另外一个套接字中读取到这个单字节数据。

fileno() 方法使用一个函数比如 select() 来让这个队列可以被轮询。
它仅仅只是暴露了底层被 get() 函数使用到的socket的文件描述符而已。

下面是一个例子，定义了一个为到来的元素监控多个队列的消费者：

In [ ]:
i
m
p
o
r
t
 
s
e
l
e
c
t


i
m
p
o
r
t
 
t
h
r
e
a
d
i
n
g




d
e
f
 
c
o
n
s
u
m
e
r
(
q
u
e
u
e
s
)
:


 
 
 
 
'
'
'


 
 
 
 
C
o
n
s
u
m
e
r
 
t
h
a
t
 
r
e
a
d
s
 
d
a
t
a
 
o
n
 
m
u
l
t
i
p
l
e
 
q
u
e
u
e
s
 
s
i
m
u
l
t
a
n
e
o
u
s
l
y


 
 
 
 
'
'
'


 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
c
a
n
_
r
e
a
d
,
 
_
,
 
_
 
=
 
s
e
l
e
c
t
.
s
e
l
e
c
t
(
q
u
e
u
e
s
,
[
]
,
[
]
)


 
 
 
 
 
 
 
 
f
o
r
 
r
 
i
n
 
c
a
n
_
r
e
a
d
:


 
 
 
 
 
 
 
 
 
 
 
 
i
t
e
m
 
=
 
r
.
g
e
t
(
)


 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
G
o
t
:
'
,
 
i
t
e
m
)




q
1
 
=
 
P
o
l
l
a
b
l
e
Q
u
e
u
e
(
)


q
2
 
=
 
P
o
l
l
a
b
l
e
Q
u
e
u
e
(
)


q
3
 
=
 
P
o
l
l
a
b
l
e
Q
u
e
u
e
(
)


t
 
=
 
t
h
r
e
a
d
i
n
g
.
T
h
r
e
a
d
(
t
a
r
g
e
t
=
c
o
n
s
u
m
e
r
,
 
a
r
g
s
=
(
[
q
1
,
q
2
,
q
3
]
,
)
)


t
.
d
a
e
m
o
n
 
=
 
T
r
u
e


t
.
s
t
a
r
t
(
)




#
 
F
e
e
d
 
d
a
t
a
 
t
o
 
t
h
e
 
q
u
e
u
e
s


q
1
.
p
u
t
(
1
)


q
2
.
p
u
t
(
1
0
)


q
3
.
p
u
t
(
'
h
e
l
l
o
'
)


q
2
.
p
u
t
(
1
5
)


.
.
.



如果你试着运行它，你会发现这个消费者会接受到所有的被放入的元素，不管元素被放进了哪个队列中。

### 讨论


对于轮询非类文件对象，比如队列通常都是比较棘手的问题。
例如，如果你不使用上面的套接字技术，
你唯一的选择就是编写代码来循环遍历这些队列并使用一个定时器。像下面这样：

In [ ]:
i
m
p
o
r
t
 
t
i
m
e


d
e
f
 
c
o
n
s
u
m
e
r
(
q
u
e
u
e
s
)
:


 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
f
o
r
 
q
 
i
n
 
q
u
e
u
e
s
:


 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
n
o
t
 
q
.
e
m
p
t
y
(
)
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
t
e
m
 
=
 
q
.
g
e
t
(
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
G
o
t
:
'
,
 
i
t
e
m
)




 
 
 
 
 
 
 
 
#
 
S
l
e
e
p
 
b
r
i
e
f
l
y
 
t
o
 
a
v
o
i
d
 
1
0
0
%
 
C
P
U


 
 
 
 
 
 
 
 
t
i
m
e
.
s
l
e
e
p
(
0
.
0
1
)



这样做其实不合理，还会引入其他的性能问题。
例如，如果新的数据被加入到一个队列中，至少要花10毫秒才能被发现。
如果你之前的轮询还要去轮询其他对象，比如网络套接字那还会有更多问题。
例如，如果你想同时轮询套接字和队列，你可能要像下面这样使用：

In [ ]:
i
m
p
o
r
t
 
s
e
l
e
c
t




d
e
f
 
e
v
e
n
t
_
l
o
o
p
(
s
o
c
k
e
t
s
,
 
q
u
e
u
e
s
)
:


 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
#
 
p
o
l
l
i
n
g
 
w
i
t
h
 
a
 
t
i
m
e
o
u
t


 
 
 
 
 
 
 
 
c
a
n
_
r
e
a
d
,
 
_
,
 
_
 
=
 
s
e
l
e
c
t
.
s
e
l
e
c
t
(
s
o
c
k
e
t
s
,
 
[
]
,
 
[
]
,
 
0
.
0
1
)


 
 
 
 
 
 
 
 
f
o
r
 
r
 
i
n
 
c
a
n
_
r
e
a
d
:


 
 
 
 
 
 
 
 
 
 
 
 
h
a
n
d
l
e
_
r
e
a
d
(
r
)


 
 
 
 
 
 
 
 
f
o
r
 
q
 
i
n
 
q
u
e
u
e
s
:


 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
n
o
t
 
q
.
e
m
p
t
y
(
)
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
t
e
m
 
=
 
q
.
g
e
t
(
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
G
o
t
:
'
,
 
i
t
e
m
)



这个方案通过将队列和套接字等同对待来解决了大部分的问题。
一个单独的 select() 调用可被同时用来轮询。
使用超时或其他基于时间的机制来执行周期性检查并没有必要。
甚至，如果数据被加入到一个队列，消费者几乎可以实时的被通知。
尽管会有一点点底层的I/O损耗，使用它通常会获得更好的响应时间并简化编程。

## 12.14 在Unix系统上面启动守护进程


### 问题


你想编写一个作为一个在Unix或类Unix系统上面运行的守护进程运行的程序。

### 解决方案


创建一个正确的守护进程需要一个精确的系统调用序列以及对于细节的控制。
下面的代码展示了怎样定义一个守护进程，可以启动后很容易的停止它。

In [ ]:
#
!
/
u
s
r
/
b
i
n
/
e
n
v
 
p
y
t
h
o
n
3


#
 
d
a
e
m
o
n
.
p
y




i
m
p
o
r
t
 
o
s


i
m
p
o
r
t
 
s
y
s




i
m
p
o
r
t
 
a
t
e
x
i
t


i
m
p
o
r
t
 
s
i
g
n
a
l




d
e
f
 
d
a
e
m
o
n
i
z
e
(
p
i
d
f
i
l
e
,
 
*
,
 
s
t
d
i
n
=
'
/
d
e
v
/
n
u
l
l
'
,


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
t
d
o
u
t
=
'
/
d
e
v
/
n
u
l
l
'
,


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
t
d
e
r
r
=
'
/
d
e
v
/
n
u
l
l
'
)
:




 
 
 
 
i
f
 
o
s
.
p
a
t
h
.
e
x
i
s
t
s
(
p
i
d
f
i
l
e
)
:


 
 
 
 
 
 
 
 
r
a
i
s
e
 
R
u
n
t
i
m
e
E
r
r
o
r
(
'
A
l
r
e
a
d
y
 
r
u
n
n
i
n
g
'
)




 
 
 
 
#
 
F
i
r
s
t
 
f
o
r
k
 
(
d
e
t
a
c
h
e
s
 
f
r
o
m
 
p
a
r
e
n
t
)


 
 
 
 
t
r
y
:


 
 
 
 
 
 
 
 
i
f
 
o
s
.
f
o
r
k
(
)
 
>
 
0
:


 
 
 
 
 
 
 
 
 
 
 
 
r
a
i
s
e
 
S
y
s
t
e
m
E
x
i
t
(
0
)
 
 
 
#
 
P
a
r
e
n
t
 
e
x
i
t


 
 
 
 
e
x
c
e
p
t
 
O
S
E
r
r
o
r
 
a
s
 
e
:


 
 
 
 
 
 
 
 
r
a
i
s
e
 
R
u
n
t
i
m
e
E
r
r
o
r
(
'
f
o
r
k
 
#
1
 
f
a
i
l
e
d
.
'
)




 
 
 
 
o
s
.
c
h
d
i
r
(
'
/
'
)


 
 
 
 
o
s
.
u
m
a
s
k
(
0
)


 
 
 
 
o
s
.
s
e
t
s
i
d
(
)


 
 
 
 
#
 
S
e
c
o
n
d
 
f
o
r
k
 
(
r
e
l
i
n
q
u
i
s
h
 
s
e
s
s
i
o
n
 
l
e
a
d
e
r
s
h
i
p
)


 
 
 
 
t
r
y
:


 
 
 
 
 
 
 
 
i
f
 
o
s
.
f
o
r
k
(
)
 
>
 
0
:


 
 
 
 
 
 
 
 
 
 
 
 
r
a
i
s
e
 
S
y
s
t
e
m
E
x
i
t
(
0
)


 
 
 
 
e
x
c
e
p
t
 
O
S
E
r
r
o
r
 
a
s
 
e
:


 
 
 
 
 
 
 
 
r
a
i
s
e
 
R
u
n
t
i
m
e
E
r
r
o
r
(
'
f
o
r
k
 
#
2
 
f
a
i
l
e
d
.
'
)




 
 
 
 
#
 
F
l
u
s
h
 
I
/
O
 
b
u
f
f
e
r
s


 
 
 
 
s
y
s
.
s
t
d
o
u
t
.
f
l
u
s
h
(
)


 
 
 
 
s
y
s
.
s
t
d
e
r
r
.
f
l
u
s
h
(
)




 
 
 
 
#
 
R
e
p
l
a
c
e
 
f
i
l
e
 
d
e
s
c
r
i
p
t
o
r
s
 
f
o
r
 
s
t
d
i
n
,
 
s
t
d
o
u
t
,
 
a
n
d
 
s
t
d
e
r
r


 
 
 
 
w
i
t
h
 
o
p
e
n
(
s
t
d
i
n
,
 
'
r
b
'
,
 
0
)
 
a
s
 
f
:


 
 
 
 
 
 
 
 
o
s
.
d
u
p
2
(
f
.
f
i
l
e
n
o
(
)
,
 
s
y
s
.
s
t
d
i
n
.
f
i
l
e
n
o
(
)
)


 
 
 
 
w
i
t
h
 
o
p
e
n
(
s
t
d
o
u
t
,
 
'
a
b
'
,
 
0
)
 
a
s
 
f
:


 
 
 
 
 
 
 
 
o
s
.
d
u
p
2
(
f
.
f
i
l
e
n
o
(
)
,
 
s
y
s
.
s
t
d
o
u
t
.
f
i
l
e
n
o
(
)
)


 
 
 
 
w
i
t
h
 
o
p
e
n
(
s
t
d
e
r
r
,
 
'
a
b
'
,
 
0
)
 
a
s
 
f
:


 
 
 
 
 
 
 
 
o
s
.
d
u
p
2
(
f
.
f
i
l
e
n
o
(
)
,
 
s
y
s
.
s
t
d
e
r
r
.
f
i
l
e
n
o
(
)
)




 
 
 
 
#
 
W
r
i
t
e
 
t
h
e
 
P
I
D
 
f
i
l
e


 
 
 
 
w
i
t
h
 
o
p
e
n
(
p
i
d
f
i
l
e
,
'
w
'
)
 
a
s
 
f
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
o
s
.
g
e
t
p
i
d
(
)
,
f
i
l
e
=
f
)




 
 
 
 
#
 
A
r
r
a
n
g
e
 
t
o
 
h
a
v
e
 
t
h
e
 
P
I
D
 
f
i
l
e
 
r
e
m
o
v
e
d
 
o
n
 
e
x
i
t
/
s
i
g
n
a
l


 
 
 
 
a
t
e
x
i
t
.
r
e
g
i
s
t
e
r
(
l
a
m
b
d
a
:
 
o
s
.
r
e
m
o
v
e
(
p
i
d
f
i
l
e
)
)




 
 
 
 
#
 
S
i
g
n
a
l
 
h
a
n
d
l
e
r
 
f
o
r
 
t
e
r
m
i
n
a
t
i
o
n
 
(
r
e
q
u
i
r
e
d
)


 
 
 
 
d
e
f
 
s
i
g
t
e
r
m
_
h
a
n
d
l
e
r
(
s
i
g
n
o
,
 
f
r
a
m
e
)
:


 
 
 
 
 
 
 
 
r
a
i
s
e
 
S
y
s
t
e
m
E
x
i
t
(
1
)




 
 
 
 
s
i
g
n
a
l
.
s
i
g
n
a
l
(
s
i
g
n
a
l
.
S
I
G
T
E
R
M
,
 
s
i
g
t
e
r
m
_
h
a
n
d
l
e
r
)




d
e
f
 
m
a
i
n
(
)
:


 
 
 
 
i
m
p
o
r
t
 
t
i
m
e


 
 
 
 
s
y
s
.
s
t
d
o
u
t
.
w
r
i
t
e
(
'
D
a
e
m
o
n
 
s
t
a
r
t
e
d
 
w
i
t
h
 
p
i
d
 
{
}
\
n
'
.
f
o
r
m
a
t
(
o
s
.
g
e
t
p
i
d
(
)
)
)


 
 
 
 
w
h
i
l
e
 
T
r
u
e
:


 
 
 
 
 
 
 
 
s
y
s
.
s
t
d
o
u
t
.
w
r
i
t
e
(
'
D
a
e
m
o
n
 
A
l
i
v
e
!
 
{
}
\
n
'
.
f
o
r
m
a
t
(
t
i
m
e
.
c
t
i
m
e
(
)
)
)


 
 
 
 
 
 
 
 
t
i
m
e
.
s
l
e
e
p
(
1
0
)




i
f
 
_
_
n
a
m
e
_
_
 
=
=
 
'
_
_
m
a
i
n
_
_
'
:


 
 
 
 
P
I
D
F
I
L
E
 
=
 
'
/
t
m
p
/
d
a
e
m
o
n
.
p
i
d
'




 
 
 
 
i
f
 
l
e
n
(
s
y
s
.
a
r
g
v
)
 
!
=
 
2
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
U
s
a
g
e
:
 
{
}
 
[
s
t
a
r
t
|
s
t
o
p
]
'
.
f
o
r
m
a
t
(
s
y
s
.
a
r
g
v
[
0
]
)
,
 
f
i
l
e
=
s
y
s
.
s
t
d
e
r
r
)


 
 
 
 
 
 
 
 
r
a
i
s
e
 
S
y
s
t
e
m
E
x
i
t
(
1
)




 
 
 
 
i
f
 
s
y
s
.
a
r
g
v
[
1
]
 
=
=
 
'
s
t
a
r
t
'
:


 
 
 
 
 
 
 
 
t
r
y
:


 
 
 
 
 
 
 
 
 
 
 
 
d
a
e
m
o
n
i
z
e
(
P
I
D
F
I
L
E
,


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
t
d
o
u
t
=
'
/
t
m
p
/
d
a
e
m
o
n
.
l
o
g
'
,


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
t
d
e
r
r
=
'
/
t
m
p
/
d
a
m
e
o
n
.
l
o
g
'
)


 
 
 
 
 
 
 
 
e
x
c
e
p
t
 
R
u
n
t
i
m
e
E
r
r
o
r
 
a
s
 
e
:


 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
e
,
 
f
i
l
e
=
s
y
s
.
s
t
d
e
r
r
)


 
 
 
 
 
 
 
 
 
 
 
 
r
a
i
s
e
 
S
y
s
t
e
m
E
x
i
t
(
1
)




 
 
 
 
 
 
 
 
m
a
i
n
(
)




 
 
 
 
e
l
i
f
 
s
y
s
.
a
r
g
v
[
1
]
 
=
=
 
'
s
t
o
p
'
:


 
 
 
 
 
 
 
 
i
f
 
o
s
.
p
a
t
h
.
e
x
i
s
t
s
(
P
I
D
F
I
L
E
)
:


 
 
 
 
 
 
 
 
 
 
 
 
w
i
t
h
 
o
p
e
n
(
P
I
D
F
I
L
E
)
 
a
s
 
f
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
o
s
.
k
i
l
l
(
i
n
t
(
f
.
r
e
a
d
(
)
)
,
 
s
i
g
n
a
l
.
S
I
G
T
E
R
M
)


 
 
 
 
 
 
 
 
e
l
s
e
:


 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
N
o
t
 
r
u
n
n
i
n
g
'
,
 
f
i
l
e
=
s
y
s
.
s
t
d
e
r
r
)


 
 
 
 
 
 
 
 
 
 
 
 
r
a
i
s
e
 
S
y
s
t
e
m
E
x
i
t
(
1
)




 
 
 
 
e
l
s
e
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
U
n
k
n
o
w
n
 
c
o
m
m
a
n
d
 
{
!
r
}
'
.
f
o
r
m
a
t
(
s
y
s
.
a
r
g
v
[
1
]
)
,
 
f
i
l
e
=
s
y
s
.
s
t
d
e
r
r
)


 
 
 
 
 
 
 
 
r
a
i
s
e
 
S
y
s
t
e
m
E
x
i
t
(
1
)



要启动这个守护进程，用户需要使用如下的命令：

In [ ]:
b
a
s
h
 
%
 
d
a
e
m
o
n
.
p
y
 
s
t
a
r
t


b
a
s
h
 
%
 
c
a
t
 
/
t
m
p
/
d
a
e
m
o
n
.
p
i
d


2
8
8
2


b
a
s
h
 
%
 
t
a
i
l
 
-
f
 
/
t
m
p
/
d
a
e
m
o
n
.
l
o
g


D
a
e
m
o
n
 
s
t
a
r
t
e
d
 
w
i
t
h
 
p
i
d
 
2
8
8
2


D
a
e
m
o
n
 
A
l
i
v
e
!
 
F
r
i
 
O
c
t
 
1
2
 
1
3
:
4
5
:
3
7
 
2
0
1
2


D
a
e
m
o
n
 
A
l
i
v
e
!
 
F
r
i
 
O
c
t
 
1
2
 
1
3
:
4
5
:
4
7
 
2
0
1
2


.
.
.



守护进程可以完全在后台运行，因此这个命令会立即返回。
不过，你可以像上面那样查看与它相关的pid文件和日志。要停止这个守护进程，使用：

In [ ]:
b
a
s
h
 
%
 
d
a
e
m
o
n
.
p
y
 
s
t
o
p


b
a
s
h
 
%



### 讨论


本节定义了一个函数 daemonize() ，在程序启动时被调用使得程序以一个守护进程来运行。
daemonize() 函数只接受关键字参数，这样的话可选参数在被使用时就更清晰了。
它会强制用户像下面这样使用它：

In [ ]:
d
a
e
m
o
n
i
z
e
(
'
d
a
e
m
o
n
.
p
i
d
'
,


 
 
 
 
 
 
 
 
 
 
s
t
d
i
n
=
'
/
d
e
v
/
n
u
l
l
,


 
 
 
 
 
 
 
 
 
 
s
t
d
o
u
t
=
'
/
t
m
p
/
d
a
e
m
o
n
.
l
o
g
'
,


 
 
 
 
 
 
 
 
 
 
s
t
d
e
r
r
=
'
/
t
m
p
/
d
a
e
m
o
n
.
l
o
g
'
)



而不是像下面这样含糊不清的调用：

In [ ]:
#
 
I
l
l
e
g
a
l
.
 
M
u
s
t
 
u
s
e
 
k
e
y
w
o
r
d
 
a
r
g
u
m
e
n
t
s


d
a
e
m
o
n
i
z
e
(
'
d
a
e
m
o
n
.
p
i
d
'
,


 
 
 
 
 
 
 
 
 
 
'
/
d
e
v
/
n
u
l
l
'
,
 
'
/
t
m
p
/
d
a
e
m
o
n
.
l
o
g
'
,
'
/
t
m
p
/
d
a
e
m
o
n
.
l
o
g
'
)



创建一个守护进程的步骤看上去不是很易懂，但是大体思想是这样的，
首先，一个守护进程必须要从父进程中脱离。
这是由 os.fork() 操作来完成的，并立即被父进程终止。

在子进程变成孤儿后，调用 os.setsid() 创建了一个全新的进程会话，并设置子进程为首领。
它会设置这个子进程为新的进程组的首领，并确保不会再有控制终端。
如果这些听上去太魔幻，因为它需要将守护进程同终端分离开并确保信号机制对它不起作用。
调用 os.chdir() 和 os.umask(0) 改变了当前工作目录并重置文件权限掩码。
修改目录通常是个好主意，因为这样可以使得它不再工作在被启动时的目录。

另外一个调用 os.fork() 在这里更加神秘点。
这一步使得守护进程失去了获取新的控制终端的能力并且让它更加独立
（本质上，该daemon放弃了它的会话首领低位，因此再也没有权限去打开控制终端了）。
尽管你可以忽略这一步，但是最好不要这么做。

一旦守护进程被正确的分离，它会重新初始化标准I/O流指向用户指定的文件。
这一部分有点难懂。跟标准I/O流相关的文件对象的引用在解释器中多个地方被找到
（sys.stdout, sys.__stdout__等）。
仅仅简单的关闭 sys.stdout 并重新指定它是行不通的，
因为没办法知道它是否全部都是用的是 sys.stdout 。
这里，我们打开了一个单独的文件对象，并调用 os.dup2() ，
用它来代替被 sys.stdout 使用的文件描述符。
这样，sys.stdout 使用的原始文件会被关闭并由新的来替换。
还要强调的是任何用于文件编码或文本处理的标准I/O流还会保留原状。

守护进程的一个通常实践是在一个文件中写入进程ID，可以被其他程序后面使用到。
daemonize() 函数的最后部分写了这个文件，但是在程序终止时删除了它。
atexit.register() 函数注册了一个函数在Python解释器终止时执行。
一个对于SIGTERM的信号处理器的定义同样需要被优雅的关闭。
信号处理器简单的抛出了 SystemExit() 异常。
或许这一步看上去没必要，但是没有它，
终止信号会使得不执行 atexit.register() 注册的清理操作的时候就杀掉了解释器。
一个杀掉进程的例子代码可以在程序最后的 stop 命令的操作中看到。

更多关于编写守护进程的信息可以查看《UNIX 环境高级编程》, 第二版
by W. Richard Stevens and Stephen A. Rago (Addison-Wesley, 2005)。
尽管它是关注与C语言编程，但是所有的内容都适用于Python，
因为所有需要的POSIX函数都可以在标准库中找到。